In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


# MRNET for Knee Diagnosis by using

1.   VGG model(implemented from scratch)
2.   ResNet model(implemented from scratch)
3.   Inception v3 model(implemented from scratch)



# Generator to load data from drive

In [ ]:
import numpy as np
import os
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from numpy import moveaxis
from numpy import asarray
import pandas as pd
import keras 

In [ ]:
class Generator(keras.utils.Sequence):
    #constructor
    def __init__(self,illness, plane, type1 ):
        # the data bath on the drive
        data_path = '/content/gdrive/My Drive/DataSet/MRNET data set/MRNet-v1.0/'
        super().__init__()
        self.illness = illness
        self.plane = plane
        self.type1 = type1

        #if the task is train or valid then its folder is ‘train’ and if the task is test then its folder is ‘valid’  

        if(type1 == "test"):
          type2 = "valid"
        if(type1 == "train"):
          type2 = "train"
        if(type1 == "valid"):
          type2 = "train"


        # get folder od exams path
        self.folder_path = data_path + type2 +"/" + plane

        #get labels path
        self.records = pd.read_csv(
              data_path +type2+"-"+illness+".csv", header=None, names=['id', 'label'])
        
        
        #creating an id for each label to be equal to its exam name (id.npy) by appending zero to its iteration until its length is 4.
        self.records['id'] = self.records['id'].map(lambda i: '0' * (4 - len(str(i))) + str(i))
        
        # return the true data
        #→ If the requirement is train( loading from the calculated  path first 1017 elements)                            
        #→ If the requirement is valid( loading from the calculated path last 113 elements)      
        #→ If the requirement is test( loading from the calculated  path ahh the elements)      

        if(type1 == "train"):
          self.paths = [self.folder_path+"/" + filename +
                      '.npy' for filename in self.records['id'].tolist()[:1017] ]
          self.labels = self.records['label'].tolist()[:1017]
          

        if(type1 == "valid"):
          self.paths = [self.folder_path+"/" + filename +
                      '.npy' for filename in self.records['id'].tolist()[1017:] ]
          self.labels = self.records['label'].tolist()[1017:]

        if(type1 == "test"):
          self.paths = [self.folder_path+"/" + filename +
                      '.npy' for filename in self.records['id'].tolist() ]
          self.labels = self.records['label'].tolist()

    #return the length of the data  
    def __len__(self):
        return len(self.paths)
    # return the exam and its label according to the index
    def __getitem__(self, index):


        array = np.load(self.paths[index])
        
        label = self.labels[index]
        array = np.stack((array,)*3, axis=0)
        
        array = np.expand_dims(array, axis=0)
        array = moveaxis(array ,1,4)
        label = [label]
        label = np.array([label])
        
        #print('\nThe exam shape: {}, It\'s label: {}\n'.format(array.shape, label[0][0]))

        return array, label

# VGG implementation from scratch

In [ ]:
from keras import models
from keras import layers
from keras.models import Model
from keras.layers.wrappers import TimeDistributed
from keras.layers import Dense, Input
from keras.layers.convolutional import Conv2D
from keras.layers import Conv2D, MaxPooling2D, Embedding, Reshape, Concatenate, Dropout
from keras.layers import GlobalAveragePooling2D
from keras.utils import plot_model 
from keras.layers import Activation

In [ ]:
# implement VGG for extraction features
def vgg_block(layer_in, n_filters, n_conv):
	# add convolutional layers
	for _ in range(n_conv):
		layer_in = TimeDistributed(Conv2D(n_filters, (3,3), padding='same', activation='relu'))(layer_in)
	# add max pooling layer
	layer_in = TimeDistributed(MaxPooling2D((2,2), strides=(2,2)))(layer_in)
	return layer_in


In [ ]:
from keras.models import load_model

# *VGG Model* 

In [ ]:
def build_model():
  visible = Input( shape=(None, 256, 256, 3) )
  # add vgg module
  layer = vgg_block(visible, 64, 2)
  # add vgg module
  layer = vgg_block(layer, 128, 2)
  # add vgg module
  layer = vgg_block(layer, 256, 3)
  layer = vgg_block(layer,512,3)
  layer = vgg_block(layer,512,3)

  # add Average pooling layer
  layer = TimeDistributed(GlobalAveragePooling2D())(layer)
  # time distributed with fully connected layer

  input2 = Input((None, 512))
  output2 = (keras.layers.pooling.GlobalMaxPooling1D())(input2)
  output2 = Model(inputs = input2  ,output = output2 )

  layer = (output2)(layer)
  layer = layers.Dense(1, activation= 'sigmoid')(layer)

  # create model
  model = Model(inputs=visible, outputs=layer)

  # compile model
  model.compile(loss = "binary_crossentropy", optimizer = "adam",metrics=['accuracy'])

  # summarize model
  model.summary()

  return model

In [ ]:
!mkdir -p saved_model

# the 9 Models using VGG 

1.   train the models
2.   test the models and get thier accuracy and loss





# --> Abnormal



1.   Axial



In [ ]:
model_abnormal_axial = build_model()
training_generator = Generator( illness = "abnormal", plane = "axial", type1 = "train")
validation_generator = Generator(illness = "abnormal", plane = "axial", type1 = "valid")

# Train model on dataset
history_abnormal_axial = model_abnormal_axial.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=True,
                    workers=6, epochs = 5)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("gl...)`


Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, 256, 256, 3) 0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, None, 256, 256, 64 1792      
_________________________________________________________________
time_distributed_2 (TimeDist (None, None, 256, 256, 64 36928     
_________________________________________________________________
time_distributed_3 (TimeDist (None, None, 128, 128, 64 0         
_________________________________________________________________
time_distributed_4 (TimeDist (None, None, 128, 128, 12 73856     
_________________________________________________________________
time_distributed_5 (TimeDist (None, None, 128, 128, 12 147584    
_________________________________________________________________
time_distributed_6 (TimeDist (None, None, 64, 64, 128) 0   

In [ ]:
model_abnormal_axial.save("/content/gdrive/My Drive/VGG_models/model_abnormal_axial.h5")

In [ ]:
model_abnormal_axial = load_model("/content/gdrive/My Drive/VGG_models/model_abnormal_axial.h5")
print(model_abnormal_axial)

In [ ]:
generator = Generator( illness = "abnormal", plane = "axial", type1 = "test")
evaluate_abnormal_axial = model_abnormal_axial.evaluate_generator( generator)
loss , acc = evaluate_abnormal_axial
print("evaluation done")
print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

evaluation done

Testing loss: 0.2018851786851883, acc: 0.7916666865348816





2.   Coronal



In [ ]:
model_abnormal_coronal =  build_model()
training_generator = Generator( illness = "abnormal", plane = "coronal", type1 = "train")
validation_generator = Generator(illness = "abnormal", plane = "coronal", type1 = "valid")


# Train model on dataset
history_abnormal_coronal = model_abnormal_coronal.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=False,
                    workers=6, epochs = 5)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("gl...)`


Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, 256, 256, 3) 0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, None, 256, 256, 64 1792      
_________________________________________________________________
time_distributed_2 (TimeDist (None, None, 256, 256, 64 36928     
_________________________________________________________________
time_distributed_3 (TimeDist (None, None, 128, 128, 64 0         
_________________________________________________________________
time_distributed_4 (TimeDist (None, None, 128, 128, 12 73856     
_________________________________________________________________
time_distributed_5 (TimeDist (None, None, 128, 128, 12 147584    
_________________________________________________________________
time_distributed_6 (TimeDist (None, None, 64, 64, 128) 0   

In [ ]:
model_abnormal_coronal.save("/content/gdrive/My Drive/VGG_models/model_abnormal_coronal.h5")

In [ ]:
model_abnormal_coronal = load_model("/content/gdrive/My Drive/VGG_models/model_abnormal_coronal.h5")
print(model_abnormal_coronal)

In [ ]:
generator = Generator( illness = "abnormal", plane = "coronal", type1 = "test")
evaluate_abnormal_coronal = model_abnormal_coronal.evaluate_generator(generator=generator)
loss , acc = evaluate_abnormal_coronal
print("evaluation done")
print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

evaluation done

Testing loss: 0.21547695994377136, acc: 0.7916666865348816





3.   Sagittal



In [ ]:
model_abnormal_sagittal =  build_model()
training_generator = Generator( illness = "abnormal", plane = "sagittal", type1 = "train")
validation_generator = Generator(illness = "abnormal", plane = "sagittal", type1 = "valid")


# Train model on dataset
history_abnormal_sagittal = model_abnormal_sagittal.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=False,
                    workers=6, epochs = 5 )

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("gl...)`


Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, None, 256, 256, 3) 0         
_________________________________________________________________
time_distributed_39 (TimeDis (None, None, 256, 256, 64 1792      
_________________________________________________________________
time_distributed_40 (TimeDis (None, None, 256, 256, 64 36928     
_________________________________________________________________
time_distributed_41 (TimeDis (None, None, 128, 128, 64 0         
_________________________________________________________________
time_distributed_42 (TimeDis (None, None, 128, 128, 12 73856     
_________________________________________________________________
time_distributed_43 (TimeDis (None, None, 128, 128, 12 147584    
_________________________________________________________________
time_distributed_44 (TimeDis (None, None, 64, 64, 128) 0   

In [ ]:
model_abnormal_sagittal.save("/content/gdrive/My Drive/VGG_models/model_abnormal_sagittal.h5")

In [ ]:
model_abnormal_sagittal = load_model("/content/gdrive/My Drive/VGG_models/model_abnormal_sagittal.h5")
print(model_abnormal_sagittal)

In [ ]:
generator = Generator( illness = "abnormal", plane = "sagittal", type1 = "test")
evaluate_abnormal_sagittal = model_abnormal_sagittal.evaluate_generator(generator=generator)
loss , acc = evaluate_abnormal_sagittal
print("evaluation done")
print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

evaluation done

Testing loss: 0.3112970292568207, acc: 0.7916666865348816



# --> ACL



1.   Axial



In [ ]:
model_acl_axial = build_model()
training_generator = Generator( illness = "acl", plane = "axial", type1 = "train")
validation_generator = Generator(illness = "acl", plane = "axial", type1 = "valid")


# Train model on dataset
history_acl_axial = model_acl_axial.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=True,
                    workers=6, epochs = 5 )

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("gl...)`


Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, None, 256, 256, 3) 0         
_________________________________________________________________
time_distributed_77 (TimeDis (None, None, 256, 256, 64 1792      
_________________________________________________________________
time_distributed_78 (TimeDis (None, None, 256, 256, 64 36928     
_________________________________________________________________
time_distributed_79 (TimeDis (None, None, 128, 128, 64 0         
_________________________________________________________________
time_distributed_80 (TimeDis (None, None, 128, 128, 12 73856     
_________________________________________________________________
time_distributed_81 (TimeDis (None, None, 128, 128, 12 147584    
_________________________________________________________________
time_distributed_82 (TimeDis (None, None, 64, 64, 128) 0  

In [ ]:
model_acl_axial.save("/content/gdrive/My Drive/VGG_models/model_acl_axial.h5")

In [ ]:
model_acl_axial = load_model("/content/gdrive/My Drive/VGG_models/model_acl_axial.h5")
print(model_acl_axial)

In [ ]:
generator = Generator( illness = "acl", plane = "axial", type1 = "test")
evaluate_acl_axial = model_acl_axial.evaluate_generator(generator=generator)
loss , acc = evaluate_acl_axial
print("evaluation done")
print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

evaluation done

Testing loss: 0.1930832862854004, acc: 0.550000011920929





2.   Coronal



In [ ]:
model_acl_coronal = build_model()
training_generator = Generator( illness = "acl", plane = "coronal", type1 = "train")
validation_generator = Generator(illness = "acl", plane = "coronal", type1 = "valid")


# Train model on dataset
history_acl_coronal = model_acl_coronal.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=True,
                    workers=6, epochs = 5 )

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("gl...)`


Model: "model_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        (None, None, 256, 256, 3) 0         
_________________________________________________________________
time_distributed_96 (TimeDis (None, None, 256, 256, 64 1792      
_________________________________________________________________
time_distributed_97 (TimeDis (None, None, 256, 256, 64 36928     
_________________________________________________________________
time_distributed_98 (TimeDis (None, None, 128, 128, 64 0         
_________________________________________________________________
time_distributed_99 (TimeDis (None, None, 128, 128, 12 73856     
_________________________________________________________________
time_distributed_100 (TimeDi (None, None, 128, 128, 12 147584    
_________________________________________________________________
time_distributed_101 (TimeDi (None, None, 64, 64, 128) 0  

In [ ]:
model_acl_coronal.save("/content/gdrive/My Drive/VGG_models/model_acl_coronal.h5")

In [ ]:
model_acl_coronal = load_model("/content/gdrive/My Drive/VGG_models/model_acl_coronal.h5")
print(model_acl_coronal)

In [ ]:
generator = Generator( illness = "acl", plane = "coronal", type1 = "test")
evaluate_acl_coronal = model_acl_coronal.evaluate_generator(generator=generator)
loss , acc = evaluate_acl_coronal
print("evaluation done")
print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

evaluation done

Testing loss: 0.20545101165771484, acc: 0.550000011920929





3.   Sagittal



In [ ]:
model_acl_sagittal = build_model()
training_generator = Generator( illness = "acl", plane = "sagittal", type1 = "train")
validation_generator = Generator(illness = "acl", plane = "sagittal", type1 = "valid")


# Train model on dataset
history_acl_sagittal = model_acl_sagittal.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=True,
                    workers=6, epochs = 5 )

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("gl...)`


Model: "model_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        (None, None, 256, 256, 3) 0         
_________________________________________________________________
time_distributed_115 (TimeDi (None, None, 256, 256, 64 1792      
_________________________________________________________________
time_distributed_116 (TimeDi (None, None, 256, 256, 64 36928     
_________________________________________________________________
time_distributed_117 (TimeDi (None, None, 128, 128, 64 0         
_________________________________________________________________
time_distributed_118 (TimeDi (None, None, 128, 128, 12 73856     
_________________________________________________________________
time_distributed_119 (TimeDi (None, None, 128, 128, 12 147584    
_________________________________________________________________
time_distributed_120 (TimeDi (None, None, 64, 64, 128) 0  

In [ ]:
model_acl_sagittal.save("/content/gdrive/My Drive/VGG_models/model_acl_sagittal.h5")

In [ ]:
model_acl_sagittal = load_model("/content/gdrive/My Drive/VGG_models/model_acl_sagittal.h5")
print(model_acl_sagittal)

In [ ]:
generator = Generator( illness = "acl", plane = "sagittal", type1 = "test")
evaluate_acl_sagittal = model_acl_sagittal.evaluate_generator(generator=generator)
loss , acc = evaluate_acl_sagittal
print("evaluation done")
print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

evaluation done

Testing loss: 0.2430194914340973, acc: 0.550000011920929



# --> ***Meniscus***









1.   Axial



In [ ]:
model_meniscus_axial = build_model()
training_generator = Generator( illness = "meniscus", plane = "axial", type1 = "train")
validation_generator = Generator(illness = "meniscus", plane = "axial", type1 = "valid")


# Train model on dataset
history_meniscus_axial = model_meniscus_axial.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=True,
                    workers=6, epochs = 5 )


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("gl...)`


Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, 256, 256, 3) 0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, None, 256, 256, 64 1792      
_________________________________________________________________
time_distributed_2 (TimeDist (None, None, 256, 256, 64 36928     
_________________________________________________________________
time_distributed_3 (TimeDist (None, None, 128, 128, 64 0         
_________________________________________________________________
time_distributed_4 (TimeDist (None, None, 128, 128, 12 73856     
_________________________________________________________________
time_distributed_5 (TimeDist (None, None, 128, 128, 12 147584    
_________________________________________________________________
time_distributed_6 (TimeDist (None, None, 64, 64, 128) 0   

In [ ]:
model_meniscus_axial.save("/content/gdrive/My Drive/VGG_models/model_meniscus_axial.h5")

In [ ]:
model_meniscus_axial = load_model("/content/gdrive/My Drive/VGG_models/model_meniscus_axial.h5")
print(model_meniscus_axial)

In [ ]:
generator = Generator( illness = "meniscus", plane = "axial", type1 = "test")
evaluate_meniscus_axial = model_meniscus_axial.evaluate_generator(generator=generator)
loss , acc = evaluate_meniscus_axial
print("evaluation done")
print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

evaluation done

Testing loss: 0.8985384702682495, acc: 0.5666666626930237





2.   Coronal



In [ ]:
model_meniscus_coronal = build_model()
training_generator = Generator( illness = "meniscus", plane = "coronal", type1 = "train")
validation_generator = Generator(illness = "meniscus", plane = "coronal", type1 = "valid")


# Train model on dataset
history_meniscus_coronal = model_meniscus_coronal.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=True,
                    workers=6, epochs = 5 )

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("gl...)`


Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, None, 256, 256, 3) 0         
_________________________________________________________________
time_distributed_20 (TimeDis (None, None, 256, 256, 64 1792      
_________________________________________________________________
time_distributed_21 (TimeDis (None, None, 256, 256, 64 36928     
_________________________________________________________________
time_distributed_22 (TimeDis (None, None, 128, 128, 64 0         
_________________________________________________________________
time_distributed_23 (TimeDis (None, None, 128, 128, 12 73856     
_________________________________________________________________
time_distributed_24 (TimeDis (None, None, 128, 128, 12 147584    
_________________________________________________________________
time_distributed_25 (TimeDis (None, None, 64, 64, 128) 0   

In [ ]:
model_meniscus_coronal.save("/content/gdrive/My Drive/VGG_models/model_meniscus_coronal.h5")

In [ ]:
model_meniscus_coronal = load_model("/content/gdrive/My Drive/VGG_models/model_meniscus_coronal.h5")
print(model_meniscus_coronal)

In [ ]:
generator = Generator( illness = "meniscus", plane = "coronal", type1 = "test")
evaluate_meniscus_coronal = model_meniscus_coronal.evaluate_generator(generator=generator)
loss , acc = evaluate_meniscus_coronal
print("evaluation done")
print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

evaluation done

Testing loss: 0.9344514608383179, acc: 0.5666666626930237





3.   Sagittal



In [ ]:
model_meniscus_sagittal = build_model()
training_generator = Generator( illness = "meniscus", plane = "sagittal", type1 = "train")
validation_generator = Generator(illness = "meniscus", plane = "sagittal", type1 = "valid")


# Train model on dataset
history_meniscus_sagittal = model_meniscus_sagittal.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=True,
                    workers=6, epochs = 5 )

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("gl...)`


Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, None, 256, 256, 3) 0         
_________________________________________________________________
time_distributed_39 (TimeDis (None, None, 256, 256, 64 1792      
_________________________________________________________________
time_distributed_40 (TimeDis (None, None, 256, 256, 64 36928     
_________________________________________________________________
time_distributed_41 (TimeDis (None, None, 128, 128, 64 0         
_________________________________________________________________
time_distributed_42 (TimeDis (None, None, 128, 128, 12 73856     
_________________________________________________________________
time_distributed_43 (TimeDis (None, None, 128, 128, 12 147584    
_________________________________________________________________
time_distributed_44 (TimeDis (None, None, 64, 64, 128) 0   

In [ ]:
model_meniscus_sagittal.save("/content/gdrive/My Drive/VGG_models/model_meniscus_sagittal.h5")

In [ ]:
model_meniscus_sagittal = load_model("/content/gdrive/My Drive/VGG_models/model_meniscus_sagittal.h5")
print(model_meniscus_sagittal)

In [ ]:
generator = Generator( illness = "meniscus", plane = "sagittal", type1 = "test")
evaluate_meniscus_sagittal = model_meniscus_sagittal.evaluate_generator(generator=generator)
loss , acc = evaluate_meniscus_sagittal
print("evaluation done")
print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

evaluation done

Testing loss: 0.9974617958068848, acc: 0.5666666626930237



# Logistic Regression Model

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation
def logistic_regression():

  model = Sequential([
      Dense(1, input_dim=(3)),
      Activation('sigmoid'),
  ])
  model.add(Dropout(1))
  model.summary()
  model.compile(loss = "binary_crossentropy", optimizer = "adam",metrics=['accuracy'])
  return model
logistic_regression()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 1)                 4         
_________________________________________________________________
activation_8 (Activation)    (None, 1)                 0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 1)                 0         
Total params: 4
Trainable params: 4
Non-trainable params: 0
_________________________________________________________________


# Get labels

In [ ]:
def load_labels(type3, type2):
  data_path = '/content/gdrive/My Drive/DataSet/MRNET data set/MRNet-v1.0/'
  if(type3 == "valid"):
    type1 = "train"
  if(type3 == "train"):
    type1 = "train"
  if(type3 == "test"):
    type1 = "valid"
  csv_path = data_path + type1+"-"+type2+".csv"
  labels = pd.read_csv(
              csv_path,header = None, names=['id', 'label'])
  if(type3 == "train"):
    labels =labels['label'].tolist()[0:1017]
  if(type3 == "valid"):
    labels =labels['label'].tolist()[1017:1130]
  if(type3 == "test"):
    labels =labels['label'].tolist()
  return labels

In [ ]:
abnormalTestLabels = load_labels('test','abnormal')
aclTestLabels = load_labels('test','acl')
meniscusTestLabels = load_labels('test','meniscus')

abnormalTrainLabels = load_labels('train','abnormal')
aclTrainLabels = load_labels('train','acl')
meniscusTrainLabels = load_labels('train','meniscus')

abnormalValidLabels = load_labels('valid','abnormal')
aclValidLabels = load_labels('valid','acl')
meniscusValidLabels = load_labels('valid','meniscus')


# Load Models and get its predictions

Load the models and get thier prediction

In [ ]:
model_abnormal_axial = load_model("/content/gdrive/My Drive/VGG_models/model_abnormal_axial.h5")
print(model_abnormal_axial)

In [ ]:
generator = Generator( illness = "abnormal", plane = "axial", type1 = "test")
predict_abnormal_axial_test = model_abnormal_axial.predict_generator(generator)

generator = Generator( illness = "abnormal", plane = "axial", type1 = "train")
predict_abnormal_axial_train = model_abnormal_axial.predict_generator(generator)

generator = Generator( illness = "abnormal", plane = "axial", type1 = "valid")
predict_abnormal_axial_valid = model_abnormal_axial.predict_generator(generator)

print("prediction done")

prediction done


In [ ]:
model_abnormal_coronal = load_model("/content/gdrive/My Drive/VGG_models/model_abnormal_coronal.h5")
print(model_abnormal_coronal)

In [ ]:
generator = Generator( illness = "abnormal", plane = "coronal", type1 = "test")
predict_abnormal_coronal_test = model_abnormal_coronal.predict_generator(generator=generator)

generator = Generator( illness = "abnormal", plane = "coronal", type1 = "train")
predict_abnormal_coronal_train = model_abnormal_coronal.predict_generator(generator=generator)

generator = Generator( illness = "abnormal", plane = "coronal", type1 = "valid")
predict_abnormal_coronal_valid = model_abnormal_coronal.predict_generator(generator=generator)

print("--> prediction done")

--> prediction done


In [ ]:
model_abnormal_sagittal = load_model("/content/gdrive/My Drive/VGG_models/model_abnormal_sagittal.h5")
print(model_abnormal_sagittal)

In [ ]:
generator = Generator( illness = "abnormal", plane = "sagittal", type1 = "test")
predict_abnormal_sagittal_test = model_abnormal_sagittal.predict_generator(generator=generator)

generator = Generator( illness = "abnormal", plane = "sagittal", type1 = "train")
predict_abnormal_sagittal_train = model_abnormal_sagittal.predict_generator(generator=generator)

generator = Generator( illness = "abnormal", plane = "sagittal", type1 = "valid")
predict_abnormal_sagittal_valid = model_abnormal_sagittal.predict_generator(generator=generator)

print("--> prediction done")

--> prediction done


In [ ]:
model_acl_axial = load_model("/content/gdrive/My Drive/VGG_models/model_acl_axial.h5")
print(model_acl_axial)

In [ ]:
generator = Generator( illness = "acl", plane = "axial", type1 = "test")
predict_acl_axial_test = model_acl_axial.predict_generator(generator=generator)

generator = Generator( illness = "acl", plane = "axial", type1 = "train")
predict_acl_axial_train = model_acl_axial.predict_generator(generator=generator)

generator = Generator( illness = "acl", plane = "axial", type1 = "valid")
predict_acl_axial_valid = model_acl_axial.predict_generator(generator=generator)

print("--> prediction done")

--> prediction done


In [ ]:
model_acl_coronal = load_model("/content/gdrive/My Drive/VGG_models/model_acl_coronal.h5")
print(model_acl_coronal)

In [ ]:
generator = Generator( illness = "acl", plane = "coronal", type1 = "test")
predict_acl_coronal_test = model_acl_coronal.predict_generator(generator=generator)

generator = Generator( illness = "acl", plane = "coronal", type1 = "train")
predict_acl_coronal_train = model_acl_coronal.predict_generator(generator=generator)

generator = Generator( illness = "acl", plane = "coronal", type1 = "valid")
predict_acl_coronal_valid = model_acl_coronal.predict_generator(generator=generator)

print("--> prediction done")

--> prediction done


In [ ]:
model_acl_sagittal = load_model("/content/gdrive/My Drive/VGG_models/model_acl_sagittal.h5")
print(model_acl_sagittal)

In [ ]:
generator = Generator( illness = "acl", plane = "sagittal", type1 = "test")
predict_acl_sagittal_test = model_acl_sagittal.predict_generator(generator=generator)

generator = Generator( illness = "acl", plane = "sagittal", type1 = "train")
predict_acl_sagittal_train = model_acl_sagittal.predict_generator(generator=generator)

generator = Generator( illness = "acl", plane = "sagittal", type1 = "valid")
predict_acl_sagittal_valid = model_acl_sagittal.predict_generator(generator=generator)

print("--> prediction done")

--> prediction done


In [ ]:
model_meniscus_axial = load_model("/content/gdrive/My Drive/VGG_models/model_meniscus_axial.h5")
print(model_meniscus_axial)

In [ ]:
generator = Generator( illness = "meniscus", plane = "axial", type1 = "test")
predict_meniscus_axial_test = model_meniscus_axial.predict_generator(generator=generator)

generator = Generator( illness = "meniscus", plane = "axial", type1 = "train")
predict_meniscus_axial_train = model_meniscus_axial.predict_generator(generator=generator)

generator = Generator( illness = "meniscus", plane = "axial", type1 = "valid")
predict_meniscus_axial_valid = model_meniscus_axial.predict_generator(generator=generator)

print("--> prediction done")

--> prediction done


In [ ]:
model_meniscus_coronal = load_model("/content/gdrive/My Drive/VGG_models/model_meniscus_coronal.h5")
print(model_meniscus_coronal)

In [ ]:
generator = Generator( illness = "meniscus", plane = "coronal", type1 = "test")
predict_meniscus_coronal_test = model_meniscus_coronal.predict_generator(generator=generator)

generator = Generator( illness = "meniscus", plane = "coronal", type1 = "train")
predict_meniscus_coronal_train = model_meniscus_coronal.predict_generator(generator=generator)

generator = Generator( illness = "meniscus", plane = "coronal", type1 = "valid")
predict_meniscus_coronal_valid = model_meniscus_coronal.predict_generator(generator=generator)

print("--> prediction done")

--> prediction done


In [ ]:
model_meniscus_sagittal = load_model("/content/gdrive/My Drive/VGG_models/model_meniscus_sagittal.h5")
print(model_meniscus_sagittal)

In [ ]:
generator = Generator( illness = "meniscus", plane = "sagittal", type1 = "test")
predict_meniscus_sagittal_test = model_meniscus_sagittal.predict_generator(generator=generator)

generator = Generator( illness = "meniscus", plane = "sagittal", type1 = "train")
predict_meniscus_sagittal_train = model_meniscus_sagittal.predict_generator(generator=generator)

generator = Generator( illness = "meniscus", plane = "sagittal", type1 = "valid")
predict_meniscus_sagittal_valid = model_meniscus_sagittal.predict_generator(generator=generator)

print("--> prediction done")

--> prediction done


# Last Abnormal Model
1.   create the model
3.   train the model
2.   test the models and get thier accuracy and loss

In [ ]:
Abnormal_model = logistic_regression()

abnormal_train_predictions = []

abnormal_train_predictions.append(predict_abnormal_axial_train)
abnormal_train_predictions.append(predict_abnormal_coronal_train)
abnormal_train_predictions.append(predict_abnormal_sagittal_train)

abnormal_train_predictions = np.transpose(abnormal_train_predictions)

#print(np.array(abnormal_train_predictions)[0].shape)

abnormal_valid_predictions = []
abnormal_valid_predictions.append(predict_abnormal_axial_valid)
abnormal_valid_predictions.append(predict_abnormal_coronal_valid)
abnormal_valid_predictions.append(predict_abnormal_sagittal_valid)

#print(np.array(abnormalValidLabels).shape)

abnormal_valid_predictions = np.transpose(abnormal_valid_predictions)


Abnormal_model.fit( np.array(abnormal_train_predictions)[0] , [abnormalTrainLabels],
                   validation_data = ( np.array(abnormal_valid_predictions)[0] , np.array(abnormalValidLabels)),
                   epochs = 50,use_multiprocessing=True, workers=6)

Abnormal_model.save("/content/gdrive/My Drive/VGG_models/Abnormal_model.h5")


Train on 1017 samples, validate on 113 samples
Epoch 1/50
1017/1017 [==============================] - 0s 132us/step - loss: 0.5752 - accuracy: 0.8083 - val_loss: 0.5672 - val_accuracy: 0.8053
Epoch 2/50
1017/1017 [==============================] - 0s 71us/step - loss: 0.5577 - accuracy: 0.8083 - val_loss: 0.5515 - val_accuracy: 0.8053
Epoch 3/50
1017/1017 [==============================] - 0s 72us/step - loss: 0.5432 - accuracy: 0.8083 - val_loss: 0.5388 - val_accuracy: 0.8053
Epoch 4/50
1017/1017 [==============================] - 0s 71us/step - loss: 0.5314 - accuracy: 0.8083 - val_loss: 0.5287 - val_accuracy: 0.8053
Epoch 5/50
1017/1017 [==============================] - 0s 71us/step - loss: 0.5222 - accuracy: 0.8083 - val_loss: 0.5201 - val_accuracy: 0.8053
Epoch 6/50
1017/1017 [==============================] - 0s 77us/step - loss: 0.5146 - accuracy: 0.8083 - val_loss: 0.5138 - val_accuracy: 0.8053
Epoch 7/50
1017/1017 [==============================] - 0s 71us/step - loss: 0.508

In [ ]:
abnormal_test_predictions = []
abnormal_test_predictions.append(predict_abnormal_axial_test)
abnormal_test_predictions.append(predict_abnormal_coronal_test)
abnormal_test_predictions.append(predict_abnormal_sagittal_test)

abnormal_test_predictions = np.transpose(abnormal_test_predictions)


loss1 , acc1 = Abnormal_model.evaluate(abnormal_test_predictions[0] , [abnormalTestLabels])
print("evaluation done")
print('\nTesting loss: {}, acc: {}\n'.format(loss1, acc1))

120/120 [==============================] - 0s 113us/step
evaluation done

Testing loss: 0.5120767186085383, acc: 0.7916666865348816



# Last ACL Model
1.   create the model
3.   train the model
2.   test the models and get thier accuracy and loss

In [ ]:
ACL_model = logistic_regression()

acl_train_predictions = []
acl_train_predictions.append(predict_acl_axial_train)
acl_train_predictions.append(predict_acl_coronal_train)
acl_train_predictions.append(predict_acl_sagittal_train)

acl_train_predictions = np.transpose(acl_train_predictions)

acl_valid_predictions = []
acl_valid_predictions.append(predict_acl_axial_valid)
acl_valid_predictions.append(predict_acl_coronal_valid)
acl_valid_predictions.append(predict_acl_sagittal_valid)

acl_valid_predictions = np.transpose(acl_valid_predictions)

ACL_model.fit(np.array(abnormal_train_predictions)[0] , [aclTrainLabels],
                   validation_data = (np.array(acl_valid_predictions)[0], [aclValidLabels]),
                   epochs = 50,use_multiprocessing=True, workers=6)

ACL_model.save("/content/gdrive/My Drive/VGG_models/ACL_model.h5")

Train on 1017 samples, validate on 113 samples
Epoch 1/50
1017/1017 [==============================] - 0s 184us/step - loss: 0.5041 - accuracy: 0.8122 - val_loss: 0.6031 - val_accuracy: 0.8496
Epoch 2/50
1017/1017 [==============================] - 0s 85us/step - loss: 0.4973 - accuracy: 0.8122 - val_loss: 0.5934 - val_accuracy: 0.8496
Epoch 3/50
1017/1017 [==============================] - 0s 87us/step - loss: 0.4928 - accuracy: 0.8122 - val_loss: 0.5848 - val_accuracy: 0.8496
Epoch 4/50
1017/1017 [==============================] - 0s 86us/step - loss: 0.4896 - accuracy: 0.8122 - val_loss: 0.5781 - val_accuracy: 0.8496
Epoch 5/50
1017/1017 [==============================] - 0s 86us/step - loss: 0.4874 - accuracy: 0.8122 - val_loss: 0.5728 - val_accuracy: 0.8496
Epoch 6/50
1017/1017 [==============================] - 0s 81us/step - loss: 0.4859 - accuracy: 0.8122 - val_loss: 0.5688 - val_accuracy: 0.8496
Epoch 7/50
1017/1017 [==============================] - 0s 71us/step - loss: 0.484

In [ ]:
acl_test_predictions = []
acl_test_predictions.append(predict_acl_axial_test)
acl_test_predictions.append(predict_acl_coronal_test)
acl_test_predictions.append(predict_acl_sagittal_test)

acl_test_predictions = np.transpose(acl_test_predictions)

loss2, acc2 = ACL_model.evaluate(acl_test_predictions[0], [aclTestLabels])
print("evaluation done")
print('\nTesting loss: {}, acc: {}\n'.format(loss2, acc2))

120/120 [==============================] - 0s 113us/step
evaluation done

Testing loss: 0.6990375399589539, acc: 0.550000011920929



# Last Meniscus Model
1.   create the model
3.   train the model
2.   test the models and get thier accuracy and loss

In [ ]:
Meniscus_model = logistic_regression()

meniscus_train_predictions = []
meniscus_train_predictions.append(predict_meniscus_axial_train)
meniscus_train_predictions.append(predict_meniscus_coronal_train)
meniscus_train_predictions.append(predict_meniscus_sagittal_train)

meniscus_train_predictions = np.transpose(meniscus_train_predictions)

meniscus_valid_predictions = []
meniscus_valid_predictions.append(predict_meniscus_axial_valid)
meniscus_valid_predictions.append(predict_meniscus_coronal_valid)
meniscus_valid_predictions.append(predict_meniscus_sagittal_valid)

meniscus_valid_predictions = np.transpose(meniscus_valid_predictions)

Meniscus_model.fit(np.array(meniscus_train_predictions)[0] , [meniscusTrainLabels],
                   validation_data = (np.array(meniscus_valid_predictions)[0], [meniscusValidLabels]),
                   epochs = 50,use_multiprocessing=True, workers=6)

Meniscus_model.save("/content/gdrive/My Drive/VGG_models/Meniscus_model.h5")

Train on 1017 samples, validate on 113 samples
Epoch 1/50
1017/1017 [==============================] - 0s 135us/step - loss: 0.6547 - accuracy: 0.6470 - val_loss: 0.6470 - val_accuracy: 0.6637
Epoch 2/50
1017/1017 [==============================] - 0s 80us/step - loss: 0.6531 - accuracy: 0.6470 - val_loss: 0.6449 - val_accuracy: 0.6637
Epoch 3/50
1017/1017 [==============================] - 0s 79us/step - loss: 0.6519 - accuracy: 0.6470 - val_loss: 0.6437 - val_accuracy: 0.6637
Epoch 4/50
1017/1017 [==============================] - 0s 84us/step - loss: 0.6511 - accuracy: 0.6470 - val_loss: 0.6425 - val_accuracy: 0.6637
Epoch 5/50
1017/1017 [==============================] - 0s 76us/step - loss: 0.6506 - accuracy: 0.6470 - val_loss: 0.6418 - val_accuracy: 0.6637
Epoch 6/50
1017/1017 [==============================] - 0s 85us/step - loss: 0.6502 - accuracy: 0.6470 - val_loss: 0.6411 - val_accuracy: 0.6637
Epoch 7/50
1017/1017 [==============================] - 0s 99us/step - loss: 0.649

In [ ]:
meniscus_test_predictions = []
meniscus_test_predictions.append(predict_meniscus_axial_test)
meniscus_test_predictions.append(predict_meniscus_coronal_test)
meniscus_test_predictions.append(predict_meniscus_sagittal_test)
meniscus_test_predictions = np.transpose(meniscus_test_predictions)
loss3, acc3 = Meniscus_model.evaluate(meniscus_test_predictions[0] ,   [meniscusTestLabels])
print("evaluation done")
print('\nTesting loss: {}, acc: {}\n'.format(loss3, acc3))

120/120 [==============================] - 0s 85us/step
evaluation done

Testing loss: 0.6979655822118124, acc: 0.5666666626930237



# Result of the first 9 Models

In [ ]:
loss_1 ,acc_1 = evaluate_abnormal_axial
loss_2 ,acc_2 = evaluate_abnormal_coronal
loss_3 ,acc_3 = evaluate_abnormal_sagittal

loss_4 ,acc_4 = evaluate_acl_axial
loss_5 ,acc_5 = evaluate_acl_coronal
loss_6 ,acc_6 = evaluate_acl_sagittal

loss_7 ,acc_7 = evaluate_meniscus_axial
loss_8 ,acc_8 = evaluate_meniscus_coronal
loss_9 ,acc_9 = evaluate_meniscus_sagittal

In [ ]:
print('\nAbnormal Axial Accuracy    : {}'.format(acc_1))
print('Abnormal Axial Loss        : {}'.format(loss_1))
print('Abnormal Coronal Accuracy  : {}'.format(acc_2))
print('Abnormal Coronal Loss      : {}'.format(loss_2))
print('Abnormal Sagittal Accuracy : {}'.format(acc_3))
print('Abnormal Sagittal Loss     : {}\n'.format(loss_3))

print('\nACL Axial Accuracy    : {}'.format(acc_4))
print('ACL Axial Loss        : {}'.format(loss_4))
print('ACL Coronal Loss      : {}'.format(acc_5))
print('ACL Coronal Accuracy  : {}'.format(loss_5))
print('ACL Sagittal Loss     : {}'.format(acc_6))
print('ACL Sagittal Accuracy : {}\n'.format(loss_6))

print('\nMeniscus Axial Accuracy    : {}'.format(acc_7))
print('Meniscus Axial Loss        : {}'.format(loss_7))
print('Meniscus Coronal Accuracy  : {}'.format(acc_8))
print('Meniscus Coronal Loss      : {}'.format(loss_8))
print('Meniscus Sagittal Accuracy : {}'.format(acc_9))
print('Meniscus Sagittal Loss     : {}\n'.format(loss_9))


Abnormal Axial Accuracy    : 0.7916666865348816
Abnormal Axial Loss        : 0.2018851786851883
Abnormal Coronal Accuracy  : 0.7916666865348816
Abnormal Coronal Loss      : 0.21547695994377136
Abnormal Sagittal Accuracy : 0.7916666865348816
Abnormal Sagittal Loss     : 0.3112970292568207


ACL Axial Accuracy    : 0.550000011920929
ACL Axial Loss        : 0.1930832862854004
ACL Coronal Loss      : 0.550000011920929
ACL Coronal Accuracy  : 0.20545101165771484
ACL Sagittal Loss     : 0.550000011920929
ACL Sagittal Accuracy : 0.2430194914340973


Meniscus Axial Accuracy    : 0.5666666626930237
Meniscus Axial Loss        : 0.8985384702682495
Meniscus Coronal Accuracy  : 0.5666666626930237
Meniscus Coronal Loss      : 0.9344514608383179
Meniscus Sagittal Accuracy : 0.5666666626930237
Meniscus Sagittal Loss     : 0.9974617958068848



# Final Accuaracy

In [ ]:
print('\nAbnormal Accuracy: {}'.format(acc1))
print('Abnormal Loss    : {}'.format(loss1))
print('\nACL Accuracy     : {}'.format(acc2))
print('ACL Loss         : {}'.format(loss2))
print('\nMeniscus Accuracy: {}'.format(acc3))
print('Meniscus Loss    : {}\n'.format(loss3))


Abnormal Accuracy: 0.7916666865348816
Abnormal Loss    : 0.5120767186085383

ACL Accuracy     : 0.550000011920929
ACL Loss         : 0.6990375399589539

Meniscus Accuracy: 0.5666666626930237
Meniscus Loss    : 0.6979655822118124



## Inception Model implementation

In [ ]:
# implement Inception for extraction features

import numpy as np
import os
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from numpy import moveaxis
from numpy import asarray
import pandas as pd
from keras import models
from keras import layers
from keras.models import Model
from keras.layers.wrappers import TimeDistributed
from keras.layers import Dense, Input
from keras.layers.convolutional import Conv2D
from keras.layers import Conv2D, MaxPooling2D, Embedding, Reshape, Concatenate, Dropout
from keras.layers import GlobalAveragePooling2D
from keras.utils import plot_model

from keras import backend as K
from keras.layers import Convolution2D, BatchNormalization, Dense
from keras.layers.core import Activation
from keras.regularizers import l2
from keras.layers.wrappers import TimeDistributed



def conv2d_bn(x, nb_filter, nb_row, nb_col,
              border_mode='same', subsample=(1, 1),
              batch_norm=True, activation='softmax',
              weight_decay=0, name=None):
    
    if name is not None:
        bn_name = 'bn_' + name
        conv_name = 'conv_' + name
    else:
        bn_name = None
        conv_name = None
    if K.common.image_dim_ordering() == 'th':
        bn_axis = 1
    else:
        bn_axis = 3

    if weight_decay and weight_decay > 0:
        x = TimeDistributed(Convolution2D(nb_filter, nb_row, nb_col,
                          subsample=subsample,
                          activation='relu',
                          W_regularizer=l2(weight_decay),
                          border_mode=border_mode,
                          name=conv_name))(x)
    else:
        x =TimeDistributed(Convolution2D(nb_filter, nb_row, nb_col,
                          subsample=subsample,
                          activation='relu',
                          border_mode=border_mode,
                          name=conv_name))(x)

    if batch_norm:
        x = BatchNormalization(axis=bn_axis, name=bn_name)(x)

    if activation:
        x = Activation(activation)(x)

    return x


from keras.layers import MaxPooling1D
from keras.models import Model
from keras.layers import Flatten, Dense, Input, BatchNormalization
from keras.layers import Convolution2D, MaxPooling2D, AveragePooling2D
from keras.engine.topology import get_source_inputs
from keras.utils.layer_utils import convert_all_kernels_in_model
from keras.utils.data_utils import get_file
from keras import backend as K
from keras.layers import Dense, GlobalAveragePooling2D
from keras.layers.core import Dropout, Lambda
import warnings
from keras.layers.merge import concatenate
Model
from keras.layers import MaxPooling2D
from keras.layers import MaxPooling1D
import tensorflow as tf
from keras.models import Sequential, load_model
from keras import models
from keras import layers
from keras.models import Model
from keras.layers.wrappers import TimeDistributed
from keras.layers import Dense, Input
from keras.layers.convolutional import Conv2D
from keras.layers import Conv2D, MaxPooling2D, Embedding, Reshape, Concatenate, Dropout
from keras.layers import GlobalAveragePooling2D
from keras.layers import GlobalMaxPooling1D


def InceptionV3(include_top=False,
                input_tensor=None, input_shape=None,
                weight_decay=0.00004, num_classes=1000,
                dropout_prob=0.,aux_include=False):
    if input_shape is None:
        input_shape = (299, 299)

    if K.common.image_dim_ordering() == 'th':
        input_shape = (3,) + input_shape
        channel_axis = 1
    else:
        input_shape = input_shape + (3,)
        channel_axis = 3
    print(input_shape)
    if input_tensor is None:
        img_input = Input(shape=input_shape)
    else:
        img_input = input_tensor

    # Using `tf` order
    # 299 x 299 x 3
    x = conv2d_bn(img_input, 32, 3, 3, subsample=(2, 2),
                  border_mode='valid', weight_decay=weight_decay,
                  name='0')

    # 149 x 149 x 32
    x = conv2d_bn(x, 32, 3, 3, border_mode='valid',
                  weight_decay=weight_decay, name='1')
    # 147 x 147 x 32
    x = conv2d_bn(x, 64, 3, 3, weight_decay=weight_decay, name='2')

    # 147 x 147 x 64
    x = TimeDistributed(MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool_1'))(x)

    # 73  x 73 x 64
    x = conv2d_bn(x, 80, 1, 1, weight_decay=weight_decay, name='3')

    # 73 x 73 x 80
    x = conv2d_bn(x, 192, 3, 3, border_mode='valid',
                  weight_decay=weight_decay, name='4')

    # 71 x 71 x 192
    x = TimeDistributed(MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool_2'))(x)

    # 35 x 35 x 192
    # Inception block
    # mixed 0: 35 x 35 x 256
    branch1x1 = conv2d_bn(x, 64, 1, 1, weight_decay=weight_decay)

    branch5x5 = conv2d_bn(x, 48, 1, 1, weight_decay=weight_decay)
    branch5x5 = conv2d_bn(branch5x5, 64, 5, 5, weight_decay=weight_decay)

    branch3x3dbl = conv2d_bn(x, 64, 1, 1, weight_decay=weight_decay)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3, weight_decay=weight_decay)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3, weight_decay=weight_decay)

    branch_pool = TimeDistributed(AveragePooling2D(
        (3, 3), strides=(1, 1), border_mode='same'))(x)
    branch_pool = conv2d_bn(branch_pool, 32, 1, 1, weight_decay=weight_decay)

    x = concatenate([branch1x1, branch5x5, branch3x3dbl, branch_pool],
              axis=4,
              name='mixed_0')


    for i in range(2):
        branch1x1 = conv2d_bn(x, 64, 1, 1, weight_decay=weight_decay)

        branch5x5 = conv2d_bn(x, 48, 1, 1, weight_decay=weight_decay)
        branch5x5 = conv2d_bn(branch5x5, 64, 5, 5, weight_decay=weight_decay)

        branch3x3dbl = conv2d_bn(x, 64, 1, 1, weight_decay=weight_decay)
        branch3x3dbl = conv2d_bn(
            branch3x3dbl, 96, 3, 3, weight_decay=weight_decay)
        branch3x3dbl = conv2d_bn(
            branch3x3dbl, 96, 3, 3, weight_decay=weight_decay)

        branch_pool = TimeDistributed(AveragePooling2D(
            (3, 3), strides=(1, 1), border_mode='same'))(x)
        branch_pool = conv2d_bn(
            branch_pool, 64, 1, 1, weight_decay=weight_decay)

        x = concatenate([branch1x1, branch5x5, branch3x3dbl, branch_pool],
                   axis=4,
                  name='mixed_' + str(i + 1))

    # mixed_3: 17 x 17 x 768
    branch3x3 = conv2d_bn(
        x, 384, 3, 3, subsample=(2, 2),
        border_mode='valid', weight_decay=weight_decay)

    branch3x3dbl = conv2d_bn(x, 64, 1, 1, weight_decay=weight_decay)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3, weight_decay=weight_decay)
    branch3x3dbl = conv2d_bn(
        branch3x3dbl, 96, 3, 3, subsample=(2, 2),
        border_mode='valid', weight_decay=weight_decay)

    branch_pool = TimeDistributed(MaxPooling2D((3, 3), strides=(2, 2), border_mode='valid'))(x)

    x = concatenate([branch3x3, branch3x3dbl, branch_pool],
              axis=4, name='mixed_3')

    # mixed_4: 17 x 17 x 768
    branch1x1 = conv2d_bn(x, 192, 1, 1, weight_decay=weight_decay)

    branch7x7 = conv2d_bn(x, 128, 1, 1, weight_decay=weight_decay)
    branch7x7 = conv2d_bn(branch7x7, 128, 1, 7, weight_decay=weight_decay)
    branch7x7 = conv2d_bn(branch7x7, 192, 7, 1, weight_decay=weight_decay)

    branch7x7dbl = conv2d_bn(x, 128, 1, 1, weight_decay=weight_decay)
    branch7x7dbl = conv2d_bn(
        branch7x7dbl, 128, 7, 1, weight_decay=weight_decay)
    branch7x7dbl = conv2d_bn(
        branch7x7dbl, 128, 1, 7, weight_decay=weight_decay)
    branch7x7dbl = conv2d_bn(
        branch7x7dbl, 128, 7, 1, weight_decay=weight_decay)
    branch7x7dbl = conv2d_bn(
        branch7x7dbl, 192, 1, 7, weight_decay=weight_decay)

    branch_pool = TimeDistributed(AveragePooling2D(
        (3, 3), strides=(1, 1), border_mode='same'))(x)
    branch_pool = conv2d_bn(branch_pool, 192, 1, 1, weight_decay=weight_decay)

    x = concatenate([branch1x1, branch7x7, branch7x7dbl, branch_pool],
               axis=4, name='mixed_4')

    # mixed 5, 6: 17 x 17 x 768
    for i in range(2):
        branch1x1 = conv2d_bn(x, 192, 1, 1, weight_decay=weight_decay)

        branch7x7 = conv2d_bn(x, 160, 1, 1, weight_decay=weight_decay)
        branch7x7 = conv2d_bn(branch7x7, 160, 1, 7, weight_decay=weight_decay)
        branch7x7 = conv2d_bn(branch7x7, 192, 7, 1, weight_decay=weight_decay)

        branch7x7dbl = conv2d_bn(x, 160, 1, 1, weight_decay=weight_decay)
        branch7x7dbl = conv2d_bn(
            branch7x7dbl, 160, 7, 1, weight_decay=weight_decay)
        branch7x7dbl = conv2d_bn(
            branch7x7dbl, 160, 1, 7, weight_decay=weight_decay)
        branch7x7dbl = conv2d_bn(
            branch7x7dbl, 160, 7, 1, weight_decay=weight_decay)
        branch7x7dbl = conv2d_bn(
            branch7x7dbl, 192, 1, 7, weight_decay=weight_decay)

        branch_pool = TimeDistributed(AveragePooling2D(
            (3, 3), strides=(1, 1), border_mode='same'))(x)
        branch_pool = conv2d_bn(
            branch_pool, 192, 1, 1, weight_decay=weight_decay)
        x = concatenate([branch1x1, branch7x7, branch7x7dbl, branch_pool],
                   axis=4,
                  name='mixed_' + str(i + 5))

    # mixed 7: 17 x 17 x 768
    branch1x1 = conv2d_bn(x, 192, 1, 1, weight_decay=weight_decay)

    branch7x7 = conv2d_bn(x, 192, 1, 1, weight_decay=weight_decay)
    branch7x7 = conv2d_bn(branch7x7, 192, 1, 7, weight_decay=weight_decay)
    branch7x7 = conv2d_bn(branch7x7, 192, 7, 1, weight_decay=weight_decay)

    branch7x7dbl = conv2d_bn(x, 192, 1, 1, weight_decay=weight_decay)
    branch7x7dbl = conv2d_bn(
        branch7x7dbl, 192, 7, 1, weight_decay=weight_decay)
    branch7x7dbl = conv2d_bn(
        branch7x7dbl, 192, 1, 7, weight_decay=weight_decay)
    branch7x7dbl = conv2d_bn(
        branch7x7dbl, 192, 7, 1, weight_decay=weight_decay)
    branch7x7dbl = conv2d_bn(
        branch7x7dbl, 192, 1, 7, weight_decay=weight_decay)

    branch_pool = TimeDistributed(AveragePooling2D(
        (3, 3), strides=(1, 1), border_mode='same'))(x)
    branch_pool = conv2d_bn(branch_pool, 192, 1, 1, weight_decay=weight_decay)

    x = concatenate([branch1x1, branch7x7, branch7x7dbl, branch_pool],
              axis=4,
              name='mixed_7')

    if aux_include:
        # Auxiliary Head logits
        aux_classifier = TimeDistributed(AveragePooling2D(
            (5, 5), strides=(3, 3), border_mode='valid'))(x)
        aux_classifier = conv2d_bn(
            aux_classifier, 128, 1, 1, weight_decay=weight_decay)

        # Shape of feature map before the final layer
        # shape = aux_classifier.output_shape
        aux_classifier = conv2d_bn(aux_classifier, 768, 5, 5,
                                   border_mode='valid',
                                   weight_decay=weight_decay)

        aux_classifier = Flatten()(aux_classifier)

        if weight_decay and weight_decay > 0:
            aux_classifier = Dense(num_classes, activation='softmax',
                                   W_regularizer=l2(weight_decay),
                                   name='aux_classifier')(aux_classifier)
        else:
            aux_classifier = Dense(
                num_classes, activation='softmax',
                name='aux_classifier')(aux_classifier)

    # mixed 8: 8 x 8 x 1280.
    branch3x3 = conv2d_bn(x, 192, 1, 1, weight_decay=weight_decay)
    branch3x3 = conv2d_bn(branch3x3, 320, 3, 3,
                          subsample=(2, 2),
                          border_mode='valid',
                          weight_decay=weight_decay)

    branch7x7x3 = conv2d_bn(x, 192, 1, 1, weight_decay=weight_decay)
    branch7x7x3 = conv2d_bn(branch7x7x3, 192, 1, 7, weight_decay=weight_decay)
    branch7x7x3 = conv2d_bn(branch7x7x3, 192, 7, 1, weight_decay=weight_decay)
    branch7x7x3 = conv2d_bn(branch7x7x3, 192, 3, 3,
                            subsample=(2, 2),
                            border_mode='valid',
                            weight_decay=weight_decay)

    branch_pool = TimeDistributed(MaxPooling2D((3, 3), strides=(2, 2), border_mode='valid'))(x)

    x = concatenate([branch3x3, branch7x7x3, branch_pool],
               axis=4,
              name='mixed_8')

    # mixed 9 10: 8 x 8 x 2048
    for i in range(2):
        branch1x1 = conv2d_bn(x, 320, 1, 1, weight_decay=weight_decay)

        branch3x3 = conv2d_bn(x, 384, 1, 1, weight_decay=weight_decay)
        branch3x3 = concatenate([conv2d_bn(branch3x3, 384, 1, 3,
                                     weight_decay=weight_decay),
                           conv2d_bn(branch3x3, 384, 3, 1,
                                     weight_decay=weight_decay)],
                          axis=4)
        branch3x3dbl = conv2d_bn(x, 448, 1, 1, weight_decay=weight_decay)
        branch3x3dbl = conv2d_bn(
            branch3x3dbl, 384, 3, 3, weight_decay=weight_decay)
        branch3x3dbl = concatenate([conv2d_bn(branch3x3dbl, 384, 1, 3,
                                        weight_decay=weight_decay),
                              conv2d_bn(branch3x3dbl, 384, 3, 1,
                                        weight_decay=weight_decay)],
                             axis=4)

        branch_pool = TimeDistributed(AveragePooling2D(
            (3, 3), strides=(1, 1), border_mode='same'))(x)
        branch_pool = conv2d_bn(
            branch_pool, 192, 1, 1, weight_decay=weight_decay)

        x = concatenate([branch1x1, branch3x3, branch3x3dbl, branch_pool],
                  axis=4,
                  name='mixed_' + str(9 + i))

    # Dimension reduction
    # 2048 x 8 x 8
    x = conv2d_bn(x, 2048, 1, 1,
                  weight_decay=weight_decay)
    
    input2 = Input((None, 2048))
    x = TimeDistributed(GlobalAveragePooling2D())(x)
    x = Dropout(dropout_prob)(x)
    y = GlobalMaxPooling1D()(input2)
    y = Model(inputs = input2  ,output = y )

    layer = (y)(x)


    # 2048
    if weight_decay and weight_decay > 0:
        layer = Dense(1,activation='softmax',W_regularizer=l2(weight_decay),name='layer')(layer)
    else:
        layer = Dense(1,activation='softmax',name='layer')(layer)
    
    if input_tensor is not None:
        inputs = get_source_inputs(input_tensor)
    else:
        inputs = img_input

    if aux_include:
        model = Model(
            inputs, [layer, aux_classifier],
            name='inception_v3_with_aux')
    else:
        model = Model(inputs, layer, name='inception_v3')
    print ('hi')

    model.summary()
    return model

In [ ]:
#build model for creating the 9 models
def get_inception_model():
 input_shape = (None,256, 256)
 x = InceptionV3(input_shape = input_shape)
 x.compile(loss='binary_crossentropy',optimizer="adam",metrics=['accuracy'])
 return x

### 9 Models for fitting and save them in drive

In [ ]:
model1 = get_inception_model()
training_generator = Generator( illness = "abnormal", plane = "coronal", type1 = "train")
validation_generator = Generator(illness = "abnormal", plane = "coronal", type1 = "valid")


# Train model on dataset
history_abnormal_coronal = model1.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=False,
                    workers=6, epochs = 5 )


In [ ]:
!mkdir -p save_inception_models
model1.save('/content/gdrive/My Drive/save_inception_models/model1.h5')

In [ ]:
model2 = get_inception_model()
training_generator = Generator( illness = "abnormal", plane = "sagittal", type1 = "train")
validation_generator = Generator(illness = "abnormal", plane = "sagittal", type1 = "valid")


# Train model on dataset
history_abnormal_sagittal = model2.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=False,
                    workers=6, epochs = 5 )

In [ ]:
!mkdir -p save_inception_models
model2.save('/content/gdrive/My Drive/save_inception_models/model2.h5')

In [ ]:
model3 = get_inception_model()
training_generator = Generator( illness = "abnormal", plane = "axial", type1 = "train")
validation_generator = Generator(illness = "abnormal", plane = "axial", type1 = "valid")


# Train model on dataset
history_abnormal_axial = model3.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=True,
                    workers=6, epochs=5)

In [ ]:
!mkdir -p save_inception_models
model3.save('/content/gdrive/My Drive/save_inception_models/model3.h5')

In [ ]:
model4 = get_inception_model()
training_generator = Generator( illness = "acl", plane = "axial", type1 = "train")
validation_generator = Generator(illness = "acl", plane = "axial", type1 = "valid")


# Train model on dataset
history_acl_axial = model4.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=False,
                    workers=6, epochs = 5 )

In [ ]:
!mkdir -p save_inception_models
model4.save('/content/gdrive/My Drive/save_inception_models/model4.h5')

In [ ]:
model5 = get_inception_model()
training_generator = Generator( illness = "acl", plane = "coronal", type1 = "train")
validation_generator = Generator(illness = "acl", plane = "coronal", type1 = "valid")


# Train model on dataset
history_acl_coronal = model5.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=False,
                    workers=6, epochs = 5 )

In [ ]:
!mkdir -p save_inception_models
model5.save('/content/gdrive/My Drive/save_inception_models/model5.h5')

In [ ]:
model6 = get_inception_model()
training_generator = Generator( illness = "acl", plane = "sagittal", type1 = "train")
validation_generator = Generator(illness = "acl", plane = "sagittal", type1 = "valid")


# Train model on dataset
history_acl_sagittal = model6.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=False,
                    workers=6, epochs = 5 )

In [ ]:
model7 = get_inception_model()
training_generator = Generator( illness = "meniscus", plane = "axial", type1 = "train")
validation_generator = Generator(illness = "meniscus", plane = "axial", type1 = "valid")


# Train model on dataset
history_meniscus_axial = model7.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=False,
                    workers=6, epochs = 5 )

In [ ]:
model8 = get_inception_model()
training_generator = Generator( illness = "meniscus", plane = "coronal", type1 = "train")
validation_generator = Generator(illness = "meniscus", plane = "coronal", type1 = "valid")


# Train model on dataset
history_meniscus_coronal = model8.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=False,
                    workers=6, epochs = 5 )

In [ ]:
model9 = get_inception_model()
training_generator = Generator( illness = "meniscus", plane = "sagittal", type1 = "train")
validation_generator = Generator(illness = "meniscus", plane = "sagittal", type1 = "valid")


# Train model on dataset
history_meniscus_sagittal = model9.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=False,
                    workers=6, epochs = 5 )

In [ ]:
abnormalTestLabels = load_labels('test','abnormal')
aclTestLabels = load_labels('test','acl')
meniscusTestLabels = load_labels('test','meniscus')

abnormalTrainLabels = load_labels('train','abnormal')
aclTrainLabels = load_labels('train','acl')
meniscusTrainLabels = load_labels('train','meniscus')

abnormalValidLabels = load_labels('valid','abnormal')
aclValidLabels = load_labels('valid','acl')
meniscusValidLabels = load_labels('valid','meniscus')


# Load Models

Load the models and get thier prediction

In [ ]:
model_abnormal_axial = load_model("/content/gdrive/My Drive/save_inception_models/model3.h5")
print(model_abnormal_axial)

In [ ]:
generator = Generator( illness = "abnormal", plane = "axial", type1 = "test")
predict_abnormal_axial_test = model_abnormal_axial.predict_generator(generator)

generator = Generator( illness = "abnormal", plane = "axial", type1 = "train")
predict_abnormal_axial_train = model_abnormal_axial.predict_generator(generator)

generator = Generator( illness = "abnormal", plane = "axial", type1 = "valid")
predict_abnormal_axial_valid = model_abnormal_axial.predict_generator(generator)

print("prediction done")

In [ ]:
model_abnormal_coronal = load_model("/content/gdrive/My Drive/save_inception_models/model1.h5")
print(model_abnormal_coronal)

In [ ]:
generator = Generator( illness = "abnormal", plane = "coronal", type1 = "test")
predict_abnormal_coronal_test = model_abnormal_coronal.predict_generator(generator=generator)

generator = Generator( illness = "abnormal", plane = "coronal", type1 = "train")
predict_abnormal_coronal_train = model_abnormal_coronal.predict_generator(generator=generator)

generator = Generator( illness = "abnormal", plane = "coronal", type1 = "valid")
predict_abnormal_coronal_valid = model_abnormal_coronal.predict_generator(generator=generator)

print("--> prediction done")

In [ ]:
model_abnormal_sagittal = load_model("/content/gdrive/My Drive/save_inception_models/model2.h5")
print(model_abnormal_sagittal)

In [ ]:
generator = Generator( illness = "abnormal", plane = "sagittal", type1 = "test")
predict_abnormal_sagittal_test = model_abnormal_sagittal.predict_generator(generator=generator)

generator = Generator( illness = "abnormal", plane = "sagittal", type1 = "train")
predict_abnormal_sagittal_train = model_abnormal_sagittal.predict_generator(generator=generator)

generator = Generator( illness = "abnormal", plane = "sagittal", type1 = "valid")
predict_abnormal_sagittal_valid = model_abnormal_sagittal.predict_generator(generator=generator)

print("--> prediction done")

In [ ]:
model_acl_axial = load_model("/content/gdrive/My Drive/save_inception_models/model4.h5")
print(model_acl_axial)

In [ ]:
generator = Generator( illness = "acl", plane = "axial", type1 = "test")
predict_acl_axial_test = model_acl_axial.predict_generator(generator=generator)

generator = Generator( illness = "acl", plane = "axial", type1 = "train")
predict_acl_axial_train = model_acl_axial.predict_generator(generator=generator)

generator = Generator( illness = "acl", plane = "axial", type1 = "valid")
predict_acl_axial_valid = model_acl_axial.predict_generator(generator=generator)

print("--> prediction done")

In [ ]:
model_acl_coronal = load_model("/content/gdrive/My Drive/save_inception_models/model5.h5")
print(model_acl_coronal)

In [ ]:
generator = Generator( illness = "acl", plane = "coronal", type1 = "test")
predict_acl_coronal_test = model_acl_coronal.predict_generator(generator=generator)

generator = Generator( illness = "acl", plane = "coronal", type1 = "train")
predict_acl_coronal_train = model_acl_coronal.predict_generator(generator=generator)

generator = Generator( illness = "acl", plane = "coronal", type1 = "valid")
predict_acl_coronal_valid = model_acl_coronal.predict_generator(generator=generator)

print("--> prediction done")

In [ ]:
model_acl_sagittal = load_model("/content/gdrive/My Drive/save_inception_models/model6.h5")
print(model_acl_sagittal)

In [ ]:
generator = Generator( illness = "acl", plane = "sagittal", type1 = "test")
predict_acl_sagittal_test = model_acl_sagittal.predict_generator(generator=generator)

generator = Generator( illness = "acl", plane = "sagittal", type1 = "train")
predict_acl_sagittal_train = model_acl_sagittal.predict_generator(generator=generator)

generator = Generator( illness = "acl", plane = "sagittal", type1 = "valid")
predict_acl_sagittal_valid = model_acl_sagittal.predict_generator(generator=generator)

print("--> prediction done")

In [ ]:
model_meniscus_axial = load_model("/content/gdrive/My Drive/save_inception_models/model7.h5")
print(model_meniscus_axial)

In [ ]:
generator = Generator( illness = "meniscus", plane = "axial", type1 = "test")
predict_meniscus_axial_test = model_meniscus_axial.predict_generator(generator=generator)

generator = Generator( illness = "meniscus", plane = "axial", type1 = "train")
predict_meniscus_axial_train = model_meniscus_axial.predict_generator(generator=generator)

generator = Generator( illness = "meniscus", plane = "axial", type1 = "valid")
predict_meniscus_axial_valid = model_meniscus_axial.predict_generator(generator=generator)

print("--> prediction done")

In [ ]:
model_meniscus_coronal = load_model("/content/gdrive/My Drive/save_inception_models/model8.h5")
print(model_meniscus_coronal)

In [ ]:
generator = Generator( illness = "meniscus", plane = "coronal", type1 = "test")
predict_meniscus_coronal_test = model_meniscus_coronal.predict_generator(generator=generator)

generator = Generator( illness = "meniscus", plane = "coronal", type1 = "train")
predict_meniscus_coronal_train = model_meniscus_coronal.predict_generator(generator=generator)

generator = Generator( illness = "meniscus", plane = "coronal", type1 = "valid")
predict_meniscus_coronal_valid = model_meniscus_coronal.predict_generator(generator=generator)

print("--> prediction done")

In [ ]:
model_meniscus_sagittal = load_model("/content/gdrive/My Drive/save_inception_models/model9.h5")
print(model_meniscus_sagittal)

In [ ]:
generator = Generator( illness = "meniscus", plane = "sagittal", type1 = "test")
predict_meniscus_sagittal_test = model_meniscus_sagittal.predict_generator(generator=generator)

generator = Generator( illness = "meniscus", plane = "sagittal", type1 = "train")
predict_meniscus_sagittal_train = model_meniscus_sagittal.predict_generator(generator=generator)

generator = Generator( illness = "meniscus", plane = "sagittal", type1 = "valid")
predict_meniscus_sagittal_valid = model_meniscus_sagittal.predict_generator(generator=generator)

print("--> prediction done")

## Evaluation of the 9 models

In [ ]:
generator = Generator( illness = "abnormal", plane = "coronal", type1 = "test")
evaluate_abnormal_coronal = model_abnormal_coronal.evaluate_generator(generator=generator)
loss , acc = evaluate_abnormal_coronal
print("evaluation done")
print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

In [ ]:
generator = Generator( illness = "abnormal", plane = "sagittal", type1 = "test")
evaluate_abnormal_sagittal = model_abnormal_sagittal.evaluate_generator(generator=generator)
loss , acc = evaluate_abnormal_sagittal
print("evaluation done")
print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

In [ ]:
generator = Generator( illness = "abnormal", plane = "axial", type1 = "test")
evaluate_abnormal_axial = model_abnormal_axial.evaluate_generator( generator)
loss , acc = evaluate_abnormal_axial
print("evaluation done")
print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

In [ ]:
generator = Generator( illness = "acl", plane = "axial", type1 = "test")
evaluate_acl_axial = model_acl_axial.evaluate_generator(generator=generator)
loss , acc = evaluate_acl_axial
print("evaluation done")
print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

In [ ]:
generator = Generator( illness = "acl", plane = "coronal", type1 = "test")
evaluate_acl_coronal = model_acl_coronal.evaluate_generator(generator=generator)
loss , acc = evaluate_acl_coronal
print("evaluation done")
print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

In [ ]:
generator = Generator( illness = "acl", plane = "sagittal", type1 = "test")
evaluate_acl_sagittal = model_acl_sagittal.evaluate_generator(generator=generator)
loss , acc = evaluate_acl_sagittal
print("evaluation done")
print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

In [ ]:
generator = Generator( illness = "meniscus", plane = "axial", type1 = "test")
evaluate_meniscus_axial = model_meniscus_axial.evaluate_generator(generator=generator)
loss , acc = evaluate_meniscus_axial
print("evaluation done")
print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

In [ ]:
generator = Generator( illness = "meniscus", plane = "coronal", type1 = "test")
evaluate_meniscus_coronal = model_meniscus_coronal.evaluate_generator(generator=generator)
loss , acc = evaluate_meniscus_coronal
print("evaluation done")
print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

In [ ]:
generator = Generator( illness = "meniscus", plane = "sagittal", type1 = "test")
evaluate_meniscus_sagittal = model_meniscus_sagittal.evaluate_generator(generator=generator)
loss , acc = evaluate_meniscus_sagittal
print("evaluation done")
print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

# Last Abnormal Model

In [ ]:
Abnormal_model = logistic_regression()

abnormal_train_predictions = []

abnormal_train_predictions.append(predict_abnormal_axial_train)
abnormal_train_predictions.append(predict_abnormal_coronal_train)
abnormal_train_predictions.append(predict_abnormal_sagittal_train)

abnormal_train_predictions = np.transpose(abnormal_train_predictions)

#print(np.array(abnormal_train_predictions)[0].shape)

abnormal_valid_predictions = []
abnormal_valid_predictions.append(predict_abnormal_axial_valid)
abnormal_valid_predictions.append(predict_abnormal_coronal_valid)
abnormal_valid_predictions.append(predict_abnormal_sagittal_valid)

#print(np.array(abnormalValidLabels).shape)

abnormal_valid_predictions = np.transpose(abnormal_valid_predictions)


Abnormal_model.fit( np.array(abnormal_train_predictions)[0] , [abnormalTrainLabels],
                   validation_data = ( np.array(abnormal_valid_predictions)[0] , np.array(abnormalValidLabels)),
                   epochs = 50,use_multiprocessing=True, workers=6)

Abnormal_model.save("/content/gdrive/My Drive/save_inception_models/Abnormal_model.h5")


In [ ]:
abnormal_test_predictions = []
abnormal_test_predictions.append(predict_abnormal_axial_test)
abnormal_test_predictions.append(predict_abnormal_coronal_test)
abnormal_test_predictions.append(predict_abnormal_sagittal_test)

abnormal_test_predictions = np.transpose(abnormal_test_predictions)


loss1 , acc1 = Abnormal_model.evaluate(abnormal_test_predictions[0] , [abnormalTestLabels])
print("evaluation done")
print('\nTesting loss: {}, acc: {}\n'.format(loss1, acc1))

# Last ACL Model

In [ ]:
ACL_model = logistic_regression()

acl_train_predictions = []
acl_train_predictions.append(predict_acl_axial_train)
acl_train_predictions.append(predict_acl_coronal_train)
acl_train_predictions.append(predict_acl_sagittal_train)

acl_train_predictions = np.transpose(acl_train_predictions)

acl_valid_predictions = []
acl_valid_predictions.append(predict_acl_axial_valid)
acl_valid_predictions.append(predict_acl_coronal_valid)
acl_valid_predictions.append(predict_acl_sagittal_valid)

acl_valid_predictions = np.transpose(acl_valid_predictions)

ACL_model.fit(np.array(acl_train_predictions)[0] , [aclTrainLabels],
                   validation_data = (np.array(acl_valid_predictions)[0], [aclValidLabels]),
                   epochs = 50,use_multiprocessing=True, workers=6)

ACL_model.save("/content/gdrive/My Drive/save_inception_models/ACL_model.h5")

In [ ]:
acl_test_predictions = []
acl_test_predictions.append(predict_acl_axial_test)
acl_test_predictions.append(predict_acl_coronal_test)
acl_test_predictions.append(predict_acl_sagittal_test)

acl_test_predictions = np.transpose(acl_test_predictions)

loss2, acc2 = ACL_model.evaluate(acl_test_predictions[0], [aclTestLabels])
print("evaluation done")
print('\nTesting loss: {}, acc: {}\n'.format(loss2, acc2))

# Last Meniscus Model

In [ ]:
Meniscus_model = logistic_regression()

meniscus_train_predictions = []
meniscus_train_predictions.append(predict_meniscus_axial_train)
meniscus_train_predictions.append(predict_meniscus_coronal_train)
meniscus_train_predictions.append(predict_meniscus_sagittal_train)

meniscus_train_predictions = np.transpose(meniscus_train_predictions)

meniscus_valid_predictions = []
meniscus_valid_predictions.append(predict_meniscus_axial_valid)
meniscus_valid_predictions.append(predict_meniscus_coronal_valid)
meniscus_valid_predictions.append(predict_meniscus_sagittal_valid)

meniscus_valid_predictions = np.transpose(meniscus_valid_predictions)

Meniscus_model.fit(np.array(meniscus_train_predictions)[0] , [meniscusTrainLabels],
                   validation_data = (np.array(meniscus_valid_predictions)[0], [meniscusValidLabels]),
                   epochs = 50,use_multiprocessing=True, workers=6)

Meniscus_model.save("/content/gdrive/My Drive/save_inception_models/Meniscus_model.h5")

In [ ]:
meniscus_test_predictions = []
meniscus_test_predictions.append(predict_meniscus_axial_test)
meniscus_test_predictions.append(predict_meniscus_coronal_test)
meniscus_test_predictions.append(predict_meniscus_sagittal_test)
meniscus_test_predictions = np.transpose(meniscus_test_predictions)
loss3, acc3 = Meniscus_model.evaluate(meniscus_test_predictions[0] ,   [meniscusTestLabels])
print("evaluation done")
print('\nTesting loss: {}, acc: {}\n'.format(loss3, acc3))

# Final Accuaracy

In [ ]:
print('\nAbnormal Accuracy: {}'.format(acc1))
print('Abnormal Loss    : {}'.format(loss1))
print('\nACL Accuracy     : {}'.format(acc2))
print('ACL Loss         : {}'.format(loss2))
print('\nMeniscus Accuracy: {}\n'.format(acc3))
print('Meniscus Loss    : {}\n'.format(loss3))

# ResNet Model Implementation

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import GlobalMaxPooling1D
import pandas as pd
import os
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from numpy import moveaxis
from numpy import asarray
import keras
from keras import models
from keras import layers
from keras.models import Model
from keras.layers.wrappers import TimeDistributed
from keras.layers import Dense, Input
from keras.layers.convolutional import Conv2D
from keras.layers import Conv2D, MaxPooling2D, Embedding, Reshape, Concatenate, Dropout
from keras.layers import GlobalAveragePooling2D
from keras.layers import GlobalAveragePooling1D
from keras.utils import plot_model 

In [ ]:
import numpy as np
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
#from resnets_utils import *
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline

import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

In [ ]:
from keras.models import Model
from keras.layers import Input
from keras.layers import Activation
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import add
from keras.utils import plot_model

def residual_module(layer_in, n_filters):
	merge_input = layer_in
	# check if the number of filters needs to be increase
	if layer_in.shape[-1] != n_filters:
		merge_input = Conv2D(n_filters, (1,1), padding='same', activation='relu', kernel_initializer='he_normal')(layer_in)
	# conv1
	conv1 = Conv2D(n_filters, (3,3), padding='same', activation='relu', kernel_initializer='he_normal')(layer_in)
	# conv2
	conv2 = Conv2D(n_filters, (3,3), padding='same', activation='linear', kernel_initializer='he_normal')(conv1)
	# add filters
	layer_out = add([conv2, merge_input])
	# activation function
	layer_out = Activation('relu')(layer_out)
	return layer_out
def build_model():
  # define model input
  visible = Input(shape=(256, 256, 3))
  layer = residual_module(visible,3)
  layer = AveragePooling2D((7,7))(layer)
  layer= Flatten()(layer)
  layer=Dense(1)(layer)
  layer=Activation("softmax")(layer)
 
  model = Model(inputs=visible, outputs=layer)
  # summarize model
  model.summary()
  return model

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
import keras 
class GeneratorR(keras.utils.Sequence):
    
    def __init__(self,illness, plane, type1 ):
        data_path = '/content/gdrive/My Drive/MRNET data set/MRNet-v1.0/' 
        super().__init__()
        self.illness = illness
        self.plane = plane
        self.type1 = type1

        if(type1 == "test"):
          type2 = "valid"
        if(type1 == "train"):
          type2 = "train"
        if(type1 == "valid"):
          type2 = "train"

        self.folder_path = data_path + type2 +"/" + plane
        self.records = pd.read_csv(
              data_path +type2+"-"+illness+".csv", header=None, names=['id', 'label'])
        
        self.records['id'] = self.records['id'].map(lambda i: '0' * (4 - len(str(i))) + str(i))
        
        if(type1 == "train"):
          self.paths = [self.folder_path+"/" + filename +
                      '.npy' for filename in self.records['id'].tolist()[:1017] ]
          self.labels = self.records['label'].tolist()[:1017]
          

        if(type1 == "valid"):
          self.paths = [self.folder_path+"/" + filename +
                      '.npy' for filename in self.records['id'].tolist()[1017:] ]
          self.labels = self.records['label'].tolist()[1017:]

        if(type1 == "test"):
          self.paths = [self.folder_path+"/" + filename +
                      '.npy' for filename in self.records['id'].tolist() ]
          self.labels = self.records['label'].tolist()
        
    def __len__(self):
        return len(self.paths)

    def __getitem__(self, index):
        array = np.load(self.paths[index])
        batch=array.shape[0]
        newarr=np.empty((batch,256,256,3))
        label=np.empty((batch),dtype=int)
        for i in range(batch):
          y=array[i]
          y=np.array(y,dtype=K.floatx())
          y=y/255.0
          y=np.expand_dims(y, axis=0)
          y=np.stack((y,)*3, axis=3)
          newarr[i]=y
          label[i]=self.labels[index]
        

        return newarr, label

Using TensorFlow backend.


# --> Abnormal


**bold text**

1.   Coronal


In [ ]:
import tensorflow as tf
model_abnormal_coronal = build_model()
model_abnormal_coronal.compile(loss=tf.keras.losses.binary_crossentropy, optimizer = "adam",metrics=["accuracy"])

training_generator = GeneratorR( illness = "abnormal", plane = "coronal", type1 = "train")
validation_generator = GeneratorR(illness = "abnormal", plane = "coronal", type1 = "valid")


# Train model on dataset
history_abnormal_coronal = model_abnormal_coronal.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=True,
                    workers=6, epochs = 10)

Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_24 (Conv2D)              (None, 256, 256, 3)  84          input_12[0][0]                   
__________________________________________________________________________________________________
conv2d_25 (Conv2D)              (None, 256, 256, 3)  84          conv2d_24[0][0]                  
__________________________________________________________________________________________________
add_8 (Add)                     (None, 256, 256, 3)  0           conv2d_25[0][0]                  
                                                                 input_12[0][0]             

/usr/local/lib/python3.6/dist-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.215045). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


1017/1017 [==============================] - 132s 130ms/step - loss: 2.9395 - accuracy: 0.8420 - val_loss: 0.0000e+00 - val_accuracy: 0.8385
Epoch 9/10
1017/1017 [==============================] - 130s 128ms/step - loss: 2.9395 - accuracy: 0.8420 - val_loss: 0.0000e+00 - val_accuracy: 0.8385
Epoch 10/10
1017/1017 [==============================] - 130s 128ms/step - loss: 2.9395 - accuracy: 0.8420 - val_loss: 0.0000e+00 - val_accuracy: 0.8385


In [ ]:
 model_abnormal_coronal.save("/content/gdrive/My Drive/resnet50_models/ model_abnormal_coronal.h5")

In [ ]:
generator = GeneratorR( illness = "abnormal", plane = "coronal", type1 = "test")
evaluate_abnormal_coronal = model_abnormal_coronal.evaluate_generator(generator=generator)
loss , acc = evaluate_abnormal_coronal
print("evaluation done")
print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

evaluation done

Testing loss: -0.0, acc: 0.8096856474876404




2.   Axial

In [ ]:
model_abnormal_axial = build_model()
model_abnormal_axial.compile(loss=tf.keras.losses.binary_crossentropy, optimizer = "adam",metrics=["accuracy"])
training_generator = GeneratorR( illness = "abnormal", plane = "axial", type1 = "train")
validation_generator = GeneratorR(illness = "abnormal", plane = "axial", type1 = "valid")

history_abnormal_axial = model_abnormal_axial.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=True,
                    workers=6, epochs = 10)

Model: "model_10"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_14 (InputLayer)           (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_28 (Conv2D)              (None, 256, 256, 3)  84          input_14[0][0]                   
__________________________________________________________________________________________________
conv2d_29 (Conv2D)              (None, 256, 256, 3)  84          conv2d_28[0][0]                  
__________________________________________________________________________________________________
add_10 (Add)                    (None, 256, 256, 3)  0           conv2d_29[0][0]                  
                                                                 input_14[0][0]            

In [ ]:
model_abnormal_axial.save("/content/gdrive/My Drive/resnet50_models/model_abnormal_axial.h5")

In [ ]:
generator = GeneratorR( illness = "abnormal", plane = "axial", type1 = "test")
evaluate_abnormal_axial = model_abnormal_axial.evaluate_generator( generator)
loss , acc = evaluate_abnormal_axial
print("evaluation done")
print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

evaluation done

Testing loss: -0.0, acc: 0.8079164624214172




3.   Sagittal


In [ ]:
model_abnormal_sagittal =  build_model()
model_abnormal_sagittal.compile(loss=tf.keras.losses.binary_crossentropy, optimizer = "adam",metrics=["accuracy"])
training_generator = GeneratorR( illness = "abnormal", plane = "sagittal", type1 = "train")
validation_generator = GeneratorR(illness = "abnormal", plane = "sagittal", type1 = "valid")

# Train model on dataset
history_abnormal_sagittal = model_abnormal_sagittal.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=False,
                    workers=6, epochs = 5 )

Model: "model_12"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_16 (InputLayer)           (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_32 (Conv2D)              (None, 256, 256, 3)  84          input_16[0][0]                   
__________________________________________________________________________________________________
conv2d_33 (Conv2D)              (None, 256, 256, 3)  84          conv2d_32[0][0]                  
__________________________________________________________________________________________________
add_12 (Add)                    (None, 256, 256, 3)  0           conv2d_33[0][0]                  
                                                                 input_16[0][0]            

In [ ]:
model_abnormal_sagittal.save("/content/gdrive/My Drive/resnet50_models/model_abnormal_sagittal.h5")

In [ ]:
generator = GeneratorR( illness = "abnormal", plane = "sagittal", type1 = "test")
evaluate_abnormal_sagittal = model_abnormal_sagittal.evaluate_generator(generator=generator)
loss , acc = evaluate_abnormal_sagittal
print("evaluation done")
print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

evaluation done

Testing loss: -0.0, acc: 0.8099918365478516



In [ ]:
model_acl_axial = build_model()
model_acl_axial.compile(loss=tf.keras.losses.binary_crossentropy, optimizer = "adam",metrics=["accuracy"])
training_generator = GeneratorR( illness = "acl", plane = "axial", type1 = "train")
validation_generator = GeneratorR(illness = "acl", plane = "axial", type1 = "valid")


# Train model on dataset
history_acl_axial = model_acl_axial.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=True,
                    workers=6, epochs = 5 )

Model: "model_13"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_17 (InputLayer)           (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_34 (Conv2D)              (None, 256, 256, 3)  84          input_17[0][0]                   
__________________________________________________________________________________________________
conv2d_35 (Conv2D)              (None, 256, 256, 3)  84          conv2d_34[0][0]                  
__________________________________________________________________________________________________
add_13 (Add)                    (None, 256, 256, 3)  0           conv2d_35[0][0]                  
                                                                 input_17[0][0]            

In [ ]:
model_acl_axial.save("/content/gdrive/My Drive/resnet50_models/model_acl_axial.h5")

In [ ]:
generator = GeneratorR( illness = "acl", plane = "axial", type1 = "test")
evaluate_acl_axial = model_acl_axial.evaluate_generator(generator=generator)
loss , acc = evaluate_acl_axial
print("evaluation done")
print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

evaluation done

Testing loss: 15.249238967895508, acc: 0.4307916462421417



In [ ]:
model_acl_coronal = build_model()
model_acl_coronal.compile(loss=tf.keras.losses.binary_crossentropy, optimizer = "adam",metrics=["accuracy"])
training_generator = GeneratorR( illness = "acl", plane = "coronal", type1 = "train")
validation_generator = GeneratorR(illness = "acl", plane = "coronal", type1 = "valid")
# Train model on dataset
history_acl_coronal = model_acl_coronal.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=True,
                    workers=6, epochs = 5 )

Model: "model_14"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_18 (InputLayer)           (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_36 (Conv2D)              (None, 256, 256, 3)  84          input_18[0][0]                   
__________________________________________________________________________________________________
conv2d_37 (Conv2D)              (None, 256, 256, 3)  84          conv2d_36[0][0]                  
__________________________________________________________________________________________________
add_14 (Add)                    (None, 256, 256, 3)  0           conv2d_37[0][0]                  
                                                                 input_18[0][0]            

In [ ]:
model_acl_coronal.save("/content/gdrive/My Drive/resnet50_models/model_acl_coronal.h5")

In [ ]:
generator = GeneratorR( illness = "acl", plane = "coronal", type1 = "test")
evaluate_acl_coronal = model_acl_coronal.evaluate_generator(generator=generator)
loss , acc = evaluate_acl_coronal
print("evaluation done")
print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

evaluation done

Testing loss: 15.249238967895508, acc: 0.4199943244457245



In [ ]:
model_acl_sagittal = build_model()
model_acl_sagittal.compile(loss=tf.keras.losses.binary_crossentropy, optimizer = "adam",metrics=["accuracy"])
training_generator = GeneratorR( illness = "acl", plane = "sagittal", type1 = "train")
validation_generator = GeneratorR(illness = "acl", plane = "sagittal", type1 = "valid")
# Train model on dataset
history_acl_sagittal = model_acl_sagittal.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=True,
                    workers=6, epochs = 5 )

Model: "model_15"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_19 (InputLayer)           (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_38 (Conv2D)              (None, 256, 256, 3)  84          input_19[0][0]                   
__________________________________________________________________________________________________
conv2d_39 (Conv2D)              (None, 256, 256, 3)  84          conv2d_38[0][0]                  
__________________________________________________________________________________________________
add_15 (Add)                    (None, 256, 256, 3)  0           conv2d_39[0][0]                  
                                                                 input_19[0][0]            

In [ ]:
model_acl_sagittal.save("/content/gdrive/My Drive/resnet50_models/model_acl_sagittal.h5")

In [ ]:
generator = GeneratorR( illness = "acl", plane = "sagittal", type1 = "test")
evaluate_acl_sagittal = model_acl_sagittal.evaluate_generator(generator=generator)
loss , acc = evaluate_acl_sagittal
print("evaluation done")
print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

evaluation done

Testing loss: 15.249238014221191, acc: 0.4395304322242737



In [ ]:
model_meniscus_axial = build_model()
model_meniscus_axial.compile(loss=tf.keras.losses.binary_crossentropy, optimizer = "adam",metrics=["accuracy"])
training_generator = GeneratorR( illness = "meniscus", plane = "axial", type1 = "train")
validation_generator = GeneratorR(illness = "meniscus", plane = "axial", type1 = "valid")
# Train model on dataset
history_meniscus_axial = model_meniscus_axial.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=True,
                    workers=6, epochs = 5 )


Model: "model_16"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_20 (InputLayer)           (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_40 (Conv2D)              (None, 256, 256, 3)  84          input_20[0][0]                   
__________________________________________________________________________________________________
conv2d_41 (Conv2D)              (None, 256, 256, 3)  84          conv2d_40[0][0]                  
__________________________________________________________________________________________________
add_16 (Add)                    (None, 256, 256, 3)  0           conv2d_41[0][0]                  
                                                                 input_20[0][0]            

/usr/local/lib/python3.6/dist-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.240802). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


1017/1017 [==============================] - 148s 146ms/step - loss: 9.9204 - accuracy: 0.3708 - val_loss: 15.2492 - val_accuracy: 0.3625
Epoch 3/5
1017/1017 [==============================] - 149s 146ms/step - loss: 9.9204 - accuracy: 0.3708 - val_loss: 15.2492 - val_accuracy: 0.3625
Epoch 4/5
1017/1017 [==============================] - 149s 147ms/step - loss: 9.9204 - accuracy: 0.3708 - val_loss: 15.2492 - val_accuracy: 0.3625
Epoch 5/5
1017/1017 [==============================] - 149s 146ms/step - loss: 9.9204 - accuracy: 0.3708 - val_loss: 15.2492 - val_accuracy: 0.3625


In [ ]:
model_meniscus_axial.save("/content/gdrive/My Drive/resnet50_models/model_meniscus_axial.h5")

In [ ]:
generator = GeneratorR( illness = "meniscus", plane = "axial", type1 = "test")
evaluate_meniscus_axial = model_meniscus_axial.evaluate_generator(generator=generator)
loss , acc = evaluate_meniscus_axial
print("evaluation done")
print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

evaluation done

Testing loss: -0.0, acc: 0.4519183933734894



In [ ]:
model_meniscus_coronal = build_model()
model_meniscus_coronal.compile(loss=tf.keras.losses.binary_crossentropy, optimizer = "adam",metrics=["accuracy"])
training_generator = GeneratorR( illness = "meniscus", plane = "coronal", type1 = "train")
validation_generator = GeneratorR(illness = "meniscus", plane = "coronal", type1 = "valid")
# Train model on dataset
history_meniscus_coronal = model_meniscus_coronal.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=True,
                    workers=6, epochs = 5 )

Model: "model_17"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_21 (InputLayer)           (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_42 (Conv2D)              (None, 256, 256, 3)  84          input_21[0][0]                   
__________________________________________________________________________________________________
conv2d_43 (Conv2D)              (None, 256, 256, 3)  84          conv2d_42[0][0]                  
__________________________________________________________________________________________________
add_17 (Add)                    (None, 256, 256, 3)  0           conv2d_43[0][0]                  
                                                                 input_21[0][0]            

In [ ]:
model_meniscus_coronal.save("/content/gdrive/My Drive/resnet50_models/model_meniscus_coronal.h5")

In [ ]:
generator = GeneratorR( illness = "meniscus", plane = "coronal", type1 = "test")
evaluate_meniscus_coronal = model_meniscus_coronal.evaluate_generator(generator=generator)
loss , acc = evaluate_meniscus_coronal
print("evaluation done")
print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

evaluation done

Testing loss: -0.0, acc: 0.4556782841682434



In [ ]:
model_meniscus_sagittal = build_model()
model_meniscus_sagittal.compile(loss=tf.keras.losses.binary_crossentropy, optimizer = "adam",metrics=["accuracy"])
training_generator = GeneratorR( illness = "meniscus", plane = "sagittal", type1 = "train")
validation_generator = GeneratorR(illness = "meniscus", plane = "sagittal", type1 = "valid")
# Train model on dataset
history_meniscus_sagittal = model_meniscus_sagittal.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=True,
                    workers=6, epochs = 5 )

Model: "model_18"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_22 (InputLayer)           (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_44 (Conv2D)              (None, 256, 256, 3)  84          input_22[0][0]                   
__________________________________________________________________________________________________
conv2d_45 (Conv2D)              (None, 256, 256, 3)  84          conv2d_44[0][0]                  
__________________________________________________________________________________________________
add_18 (Add)                    (None, 256, 256, 3)  0           conv2d_45[0][0]                  
                                                                 input_22[0][0]            

In [ ]:
model_meniscus_sagittal.save("/content/gdrive/My Drive/resnet50_models/model_meniscus_sagittal.h5")

In [ ]:
generator = GeneratorR( illness = "meniscus", plane = "sagittal", type1 = "test")
evaluate_meniscus_sagittal = model_meniscus_sagittal.evaluate_generator(generator=generator)
loss , acc = evaluate_meniscus_sagittal
print("evaluation done")
print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

evaluation done

Testing loss: -0.0, acc: 0.45945945382118225

evaluation done

Testing loss: -0.0, acc: 0.45945945382118225



In [ ]:
abnormalTestLabels = load_labels('test','abnormal')
aclTestLabels = load_labels('test','acl')
meniscusTestLabels = load_labels('test','meniscus')

abnormalTrainLabels = load_labels('train','abnormal')
aclTrainLabels = load_labels('train','acl')
meniscusTrainLabels = load_labels('train','meniscus')

abnormalValidLabels = load_labels('valid','abnormal')
aclValidLabels = load_labels('valid','acl')
meniscusValidLabels = load_labels('valid','meniscus')


In [ ]:
model_abnormal_axial = load_model("/content/gdrive/My Drive/resnet50_models/model_abnormal_axial.h5")
print(model_abnormal_axial)

In [ ]:
generator = GeneratorR( illness = "abnormal", plane = "axial", type1 = "test")
predict_abnormal_axial_test = model_abnormal_axial.predict_generator(generator)

generator = GeneratorR( illness = "abnormal", plane = "axial", type1 = "train")
predict_abnormal_axial_train = model_abnormal_axial.predict_generator(generator)

generator = GeneratorR( illness = "abnormal", plane = "axial", type1 = "valid")
predict_abnormal_axial_valid = model_abnormal_axial.predict_generator(generator)

print("prediction done")

In [ ]:
model_abnormal_coronal = load_model("/content/gdrive/My Drive/resnet50_models/ model_abnormal_coronal.h5")
print(model_abnormal_coronal)

In [ ]:
generator = GeneratorR( illness = "abnormal", plane = "coronal", type1 = "test")
predict_abnormal_coronal_test = model_abnormal_coronal.predict_generator(generator=generator)

generator = GeneratorR( illness = "abnormal", plane = "coronal", type1 = "train")
predict_abnormal_coronal_train = model_abnormal_coronal.predict_generator(generator=generator)

generator = GeneratorR( illness = "abnormal", plane = "coronal", type1 = "valid")
predict_abnormal_coronal_valid = model_abnormal_coronal.predict_generator(generator=generator)

print("--> prediction done")

--> prediction done


In [ ]:
model_abnormal_sagittal = load_model("/content/gdrive/My Drive/resnet50_models/model_abnormal_sagittal.h5")
print(model_abnormal_sagittal)

In [ ]:
generator = GeneratorR( illness = "abnormal", plane = "sagittal", type1 = "test")
predict_abnormal_sagittal_test = model_abnormal_sagittal.predict_generator(generator=generator)

generator = GeneratorR( illness = "abnormal", plane = "sagittal", type1 = "train")
predict_abnormal_sagittal_train = model_abnormal_sagittal.predict_generator(generator=generator)

generator = GeneratorR( illness = "abnormal", plane = "sagittal", type1 = "valid")
predict_abnormal_sagittal_valid = model_abnormal_sagittal.predict_generator(generator=generator)

print("--> prediction done")

--> prediction done


In [ ]:
model_acl_axial = load_model("/content/gdrive/My Drive/resnet50_models/model_acl_axial.h5")
print(model_acl_axial)

In [ ]:
generator = GeneratorR( illness = "acl", plane = "axial", type1 = "test")
predict_acl_axial_test = model_acl_axial.predict_generator(generator=generator)

generator = GeneratorR( illness = "acl", plane = "axial", type1 = "train")
predict_acl_axial_train = model_acl_axial.predict_generator(generator=generator)

generator = GeneratorR( illness = "acl", plane = "axial", type1 = "valid")
predict_acl_axial_valid = model_acl_axial.predict_generator(generator=generator)

print("--> prediction done")

--> prediction done


In [ ]:
model_acl_coronal = load_model("/content/gdrive/My Drive/resnet50_models/model_acl_coronal.h5")
print(model_acl_coronal)

In [ ]:
generator = GeneratorR( illness = "acl", plane = "coronal", type1 = "test")
predict_acl_coronal_test = model_acl_coronal.predict_generator(generator=generator)

generator = GeneratorR( illness = "acl", plane = "coronal", type1 = "train")
predict_acl_coronal_train = model_acl_coronal.predict_generator(generator=generator)

generator = GeneratorR( illness = "acl", plane = "coronal", type1 = "valid")
predict_acl_coronal_valid = model_acl_coronal.predict_generator(generator=generator)

print("--> prediction done")

--> prediction done


In [ ]:
model_acl_sagittal = load_model("/content/gdrive/My Drive/resnet50_models/model_acl_sagittal.h5")
print(model_acl_sagittal)

In [ ]:
generator = GeneratorR( illness = "acl", plane = "sagittal", type1 = "test")
predict_acl_sagittal_test = model_acl_sagittal.predict_generator(generator=generator)

generator = GeneratorR( illness = "acl", plane = "sagittal", type1 = "train")
predict_acl_sagittal_train = model_acl_sagittal.predict_generator(generator=generator)

generator = GeneratorR( illness = "acl", plane = "sagittal", type1 = "valid")
predict_acl_sagittal_valid = model_acl_sagittal.predict_generator(generator=generator)

print("--> prediction done")

--> prediction done


In [ ]:
model_meniscus_axial = load_model("/content/gdrive/My Drive/resnet50_models/model_meniscus_axial.h5")
print(model_meniscus_axial)

In [ ]:
generator = GeneratorR( illness = "meniscus", plane = "axial", type1 = "test")
predict_meniscus_axial_test = model_meniscus_axial.predict_generator(generator=generator)

generator = GeneratorR( illness = "meniscus", plane = "axial", type1 = "train")
predict_meniscus_axial_train = model_meniscus_axial.predict_generator(generator=generator)

generator = GeneratorR( illness = "meniscus", plane = "axial", type1 = "valid")
predict_meniscus_axial_valid = model_meniscus_axial.predict_generator(generator=generator)

print("--> prediction done")

--> prediction done


In [ ]:
model_meniscus_coronal = load_model("/content/gdrive/My Drive/resnet50_models/model_meniscus_coronal.h5")
print(model_meniscus_coronal)

In [ ]:
generator = GeneratorR( illness = "meniscus", plane = "coronal", type1 = "test")
predict_meniscus_coronal_test = model_meniscus_coronal.predict_generator(generator=generator)

generator = GeneratorR( illness = "meniscus", plane = "coronal", type1 = "train")
predict_meniscus_coronal_train = model_meniscus_coronal.predict_generator(generator=generator)

generator = GeneratorR( illness = "meniscus", plane = "coronal", type1 = "valid")
predict_meniscus_coronal_valid = model_meniscus_coronal.predict_generator(generator=generator)

print("--> prediction done")

--> prediction done


In [ ]:
model_meniscus_sagittal = load_model("/content/gdrive/My Drive/resnet50_models/model_meniscus_sagittal.h5")
print(model_meniscus_sagittal)

In [ ]:
generator = GeneratorR( illness = "meniscus", plane = "sagittal", type1 = "test")
predict_meniscus_sagittal_test = model_meniscus_sagittal.predict_generator(generator=generator)

generator = GeneratorR( illness = "meniscus", plane = "sagittal", type1 = "train")
predict_meniscus_sagittal_train = model_meniscus_sagittal.predict_generator(generator=generator)

generator = GeneratorR( illness = "meniscus", plane = "sagittal", type1 = "valid")
predict_meniscus_sagittal_valid = model_meniscus_sagittal.predict_generator(generator=generator)

print("--> prediction done")

--> prediction done


In [ ]:
Abnormal_model = logistic_regression()

abnormal_train_predictions = []

abnormal_train_predictions.append(predict_abnormal_axial_train)
abnormal_train_predictions.append(predict_abnormal_coronal_train)
abnormal_train_predictions.append(predict_abnormal_sagittal_train)

abnormal_train_predictions = np.transpose(abnormal_train_predictions)

#print(np.array(abnormal_train_predictions)[0].shape)

abnormal_valid_predictions = []
abnormal_valid_predictions.append(predict_abnormal_axial_valid)
abnormal_valid_predictions.append(predict_abnormal_coronal_valid)
abnormal_valid_predictions.append(predict_abnormal_sagittal_valid)

#print(np.array(abnormalValidLabels).shape)

abnormal_valid_predictions = np.transpose(abnormal_valid_predictions)


Abnormal_model.fit( np.array(abnormal_train_predictions)[0] , [abnormalTrainLabels],
                   validation_data = ( np.array(abnormal_valid_predictions)[0] , np.array(abnormalValidLabels)),
                   epochs = 50,use_multiprocessing=True, workers=6)



In [ ]:
abnormal_test_predictions = []
abnormal_test_predictions.append(predict_abnormal_axial_test)
abnormal_test_predictions.append(predict_abnormal_coronal_test)
abnormal_test_predictions.append(predict_abnormal_sagittal_test)

abnormal_test_predictions = np.transpose(abnormal_test_predictions)


loss1 , acc1 = Abnormal_model.evaluate(abnormal_test_predictions[0] , [abnormalTestLabels])
print("evaluation done")
print('\nTesting loss: {}, acc: {}\n'.format(loss1, acc1))

In [ ]:
ACL_model = logistic_regression()

acl_train_predictions = []
acl_train_predictions.append(predict_acl_axial_train)
acl_train_predictions.append(predict_acl_coronal_train)
acl_train_predictions.append(predict_acl_sagittal_train)

acl_train_predictions = np.transpose(acl_train_predictions)

acl_valid_predictions = []
acl_valid_predictions.append(predict_acl_axial_valid)
acl_valid_predictions.append(predict_acl_coronal_valid)
acl_valid_predictions.append(predict_acl_sagittal_valid)

acl_valid_predictions = np.transpose(acl_valid_predictions)

ACL_model.fit(np.array(abnormal_train_predictions)[0] , [aclTrainLabels],
                   validation_data = (np.array(acl_valid_predictions)[0], [aclValidLabels]),
                   epochs = 50,use_multiprocessing=True, workers=6)



In [ ]:
acl_test_predictions = []
acl_test_predictions.append(predict_acl_axial_test)
acl_test_predictions.append(predict_acl_coronal_test)
acl_test_predictions.append(predict_acl_sagittal_test)

acl_test_predictions = np.transpose(acl_test_predictions)

loss2, acc2 = ACL_model.evaluate(acl_test_predictions[0], [aclTestLabels])
print("evaluation done")
print('\nTesting loss: {}, acc: {}\n'.format(loss2, acc2))

In [ ]:
Meniscus_model = logistic_regression()

meniscus_train_predictions = []
meniscus_train_predictions.append(predict_meniscus_axial_train)
meniscus_train_predictions.append(predict_meniscus_coronal_train)
meniscus_train_predictions.append(predict_meniscus_sagittal_train)

meniscus_train_predictions = np.transpose(meniscus_train_predictions)

meniscus_valid_predictions = []
meniscus_valid_predictions.append(predict_meniscus_axial_valid)
meniscus_valid_predictions.append(predict_meniscus_coronal_valid)
meniscus_valid_predictions.append(predict_meniscus_sagittal_valid)

meniscus_valid_predictions = np.transpose(meniscus_valid_predictions)

Meniscus_model.fit(np.array(meniscus_train_predictions)[0] , [meniscusTrainLabels],
                   validation_data = (np.array(meniscus_valid_predictions)[0], [meniscusValidLabels]),
                   epochs = 50,use_multiprocessing=True, workers=6)



In [ ]:
meniscus_test_predictions = []
meniscus_test_predictions.append(predict_meniscus_axial_test)
meniscus_test_predictions.append(predict_meniscus_coronal_test)
meniscus_test_predictions.append(predict_meniscus_sagittal_test)
meniscus_test_predictions = np.transpose(meniscus_test_predictions)
loss3, acc3 = Meniscus_model.evaluate(meniscus_test_predictions[0] ,   [meniscusTestLabels])
print("evaluation done")
print('\nTesting loss: {}, acc: {}\n'.format(loss3, acc3))

# Final Accuaracy


In [ ]:
print('\nAbnormal Accuracy: {}'.format(acc1))
print('Abnormal Loss    : {}'.format(loss1))
print('\nACL Accuracy     : {}'.format(acc2))
print('ACL Loss         : {}'.format(loss2))
print('\nMeniscus Accuracy: {}'.format(acc3))
print('Meniscus Loss    : {}\n'.format(loss3))

# Transfer learning

In [ ]:
from keras import models
from keras import layers
from keras.models import Model
from keras.layers.wrappers import TimeDistributed
from keras.layers import Dense, Input
from keras.layers.convolutional import Conv2D
from keras.layers import Conv2D, MaxPooling2D, Embedding, Reshape, Concatenate, Dropout
from keras.layers import GlobalAveragePooling2D
from keras.utils import plot_model 
from keras.layers import Activation

In [ ]:
from keras.models import Sequential,load_model
from keras.models import Model
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard
from keras import optimizers, losses, activations, models
from keras.layers import Convolution2D, Dense, Input, Flatten, Dropout, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D, Concatenate,GlobalMaxPooling1D
from keras.applications.vgg16 import VGG16
def get_model():
 ip = Input(shape=(None,256,256,3))
 ##built in model
 def create_base():
  base_model = VGG16(weights='imagenet', 
                                include_top=False,input_shape=(256,256,3))
  layer = (GlobalAveragePooling2D())(base_model.output)
  f_model = Model(base_model.input, layer)
  return f_model

 temp_base = create_base()
##freeze outer layer##
 for layer in temp_base.layers:
     layer.trainable = False

 layer = TimeDistributed(temp_base)(ip)    
 ##add max pooling layer
 input2 = Input((None, 512))
 output2 = (keras.layers.pooling.GlobalMaxPooling1D())(input2)
 output2 = Model(inputs = input2  ,output = output2 )
 layer = (output2)(layer)
 #fully connected layer
 layer = layers.Dense(1, activation= 'sigmoid')(layer)
 model = Model(ip,layer)

 model.compile(loss = "binary_crossentropy", optimizer = "adam",metrics=['accuracy'])
 model.summary()
 return model


'\n\nadd_model = keras.Sequential()\nadd_model.add(base_model)\n\n\n\n\ninput2 = Input((None, 2048))\noutput2 = (GlobalMaxPooling1D())(input2)\noutput2 = Model(inputs = input2  ,output = output2 )\nlayer = (output2)(layer)\n\nmodel = Model(inputs=visible, outputs=layer)\n#dd_model.add(layer)\n#model = add_model\n  # compile model\nmodel.compile(loss = "binary_crossentropy", optimizer = "adam",metrics=[\'accuracy\'])\n\n  # summarize model\nmodel.summary()\n '

**Fitting and evaluating the models**

In [ ]:
model1 = get_model()
training_generator = Generator( illness = "abnormal", plane = "coronal", type1 = "train")
validation_generator = Generator(illness = "abnormal", plane = "coronal", type1 = "valid")

history_abnormal_axial = model1.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=True,
                    workers=6, epochs = 5)
model1.save('/content/gdrive/My Drive/transferVGG_models/model1.h5')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("gl...)`


Model: "model_39"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_37 (InputLayer)        (None, None, 256, 256, 3) 0         
_________________________________________________________________
time_distributed_13 (TimeDis (None, None, 512)         14714688  
_________________________________________________________________
model_38 (Model)             (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 513       
Total params: 14,715,201
Trainable params: 513
Non-trainable params: 14,714,688
_________________________________________________________________
Epoch 1/5
1017/1017 [==============================] - 133s 130ms/step - loss: 0.9370 - accuracy: 0.7424 - val_loss: 0.0783 - val_accuracy: 0.7522
Epoch 2/5
1017/1017 [==============================] - 132s 130ms/step - loss: 0.6722 - accurac

In [ ]:
generator = Generator( illness = "abnormal", plane = "coronal", type1 = "test")
evaluate_abnormal_coronal = model1.evaluate_generator(generator=generator)
loss , acc = evaluate_abnormal_coronal
print("evaluation done")
print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

evaluation done

Testing loss: 1.3666744232177734, acc: 0.824999988079071



In [ ]:
model2 = get_model()
training_generator = Generator( illness = "abnormal", plane = "sagittal", type1 = "train")
validation_generator = Generator(illness = "abnormal", plane = "sagittal", type1 = "valid")


# Train model on dataset
history_abnormal_sagittal = model2.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=False,
                    workers=6, epochs = 5 )
model2.save('/content/gdrive/My Drive/transferVGG_models/model2.h5')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("gl...)`


Model: "model_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        (None, None, 256, 256, 3) 0         
_________________________________________________________________
time_distributed_5 (TimeDist (None, None, 512)         14714688  
_________________________________________________________________
model_14 (Model)             (None, 512)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 513       
Total params: 14,715,201
Trainable params: 513
Non-trainable params: 14,714,688
_________________________________________________________________
Epoch 1/5
1017/1017 [==============================] - 140s 138ms/step - loss: 0.9423 - accuracy: 0.7561 - val_loss: 2.8959e-07 - val_accuracy: 0.7965
Epoch 2/5
1017/1017 [==============================] - 138s 135ms/step - loss: 0.6358 - acc

In [ ]:
generator = Generator( illness = "abnormal", plane = "sagittal", type1 = "test")
evaluate_abnormal_sagittal = model2.evaluate_generator(generator=generator)
loss , acc = evaluate_abnormal_sagittal
print("evaluation done")
print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

evaluation done

Testing loss: 0.0042292033322155476, acc: 0.8416666388511658



In [ ]:
model3 = get_model()
training_generator = Generator( illness = "abnormal", plane = "axial", type1 = "train")
validation_generator = Generator(illness = "abnormal", plane = "axial", type1 = "valid")


# Train model on dataset
history_abnormal_axial = model3.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=True,
                    workers=6, epochs=5)
model3.save('/content/gdrive/My Drive/transferVGG_models/model3.h5')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("gl...)`


Model: "model_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_16 (InputLayer)        (None, None, 256, 256, 3) 0         
_________________________________________________________________
time_distributed_6 (TimeDist (None, None, 512)         14714688  
_________________________________________________________________
model_17 (Model)             (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 513       
Total params: 14,715,201
Trainable params: 513
Non-trainable params: 14,714,688
_________________________________________________________________
Epoch 1/5
1016/1017 [============================>.] - ETA: 0s - loss: 0.9026 - accuracy: 0.7677

1017/1017 [==============================] - 148s 146ms/step - loss: 0.9018 - accuracy: 0.7679 - val_loss: 1.7326e-09 - val_accuracy: 0.8053
Epo

In [ ]:
generator = Generator( illness = "abnormal", plane = "axial", type1 = "test")
evaluate_abnormal_axial = model3.evaluate_generator( generator)
loss , acc = evaluate_abnormal_axial
print("evaluation done")
print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

evaluation done

Testing loss: 4.429042339324951, acc: 0.8583333492279053



In [ ]:
model4 = get_model()
training_generator = Generator( illness = "acl", plane = "axial", type1 = "train")
validation_generator = Generator(illness = "acl", plane = "axial", type1 = "valid")


# Train model on dataset
history_acl_axial = model4.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=False,
                    workers=6, epochs = 5 )
model4.save('/content/gdrive/My Drive/transferVGG_models/model4.h5')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("gl...)`


Model: "model_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_19 (InputLayer)        (None, None, 256, 256, 3) 0         
_________________________________________________________________
time_distributed_7 (TimeDist (None, None, 512)         14714688  
_________________________________________________________________
model_20 (Model)             (None, 512)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 513       
Total params: 14,715,201
Trainable params: 513
Non-trainable params: 14,714,688
_________________________________________________________________
Epoch 1/5
1017/1017 [==============================] - 153s 150ms/step - loss: 1.3864 - accuracy: 0.7080 - val_loss: 0.0068 - val_accuracy: 0.6460
Epoch 2/5
1017/1017 [==============================] - 153s 150ms/step - loss: 0.9164 - accurac

In [ ]:
generator = Generator( illness = "acl", plane = "axial", type1 = "test")
evaluate_acl_axial = model4.evaluate_generator(generator=generator)
loss , acc = evaluate_acl_axial
print("evaluation done")
print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

evaluation done

Testing loss: 0.1787252575159073, acc: 0.550000011920929



In [ ]:
model5 = get_model()
training_generator = Generator( illness = "acl", plane = "coronal", type1 = "train")
validation_generator = Generator(illness = "acl", plane = "coronal", type1 = "valid")


# Train model on dataset
history_acl_coronal = model5.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=False,
                    workers=6, epochs = 5 )
model5.save('/content/gdrive/My Drive/transferVGG_models/model5.h5')

In [ ]:
#model5 = model_load('/content/gdrive/My Drive/transferVGG_models/model5.h5')
generator = Generator( illness = "acl", plane = "coronal", type1 = "test")
evaluate_acl_coronal = model5.evaluate_generator(generator=generator)
loss , acc = evaluate_acl_coronal
print("evaluation done")
print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

evaluation done

Testing loss: 0.7524604797363281, acc: 0.5166666507720947



In [ ]:
model6 = get_model()
training_generator = Generator( illness = "acl", plane = "sagittal", type1 = "train")
validation_generator = Generator(illness = "acl", plane = "sagittal", type1 = "valid")


# Train model on dataset
history_acl_sagittal = model6.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=False,
                    workers=6, epochs = 5 )
model6.save('/content/gdrive/My Drive/transferVGG_models/model6.h5')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("gl...)`


Model: "model_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_25 (InputLayer)        (None, None, 256, 256, 3) 0         
_________________________________________________________________
time_distributed_9 (TimeDist (None, None, 512)         14714688  
_________________________________________________________________
model_26 (Model)             (None, 512)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 513       
Total params: 14,715,201
Trainable params: 513
Non-trainable params: 14,714,688
_________________________________________________________________
Epoch 1/5
1017/1017 [==============================] - 139s 137ms/step - loss: 1.0386 - accuracy: 0.7512 - val_loss: 9.4163 - val_accuracy: 0.8496
Epoch 2/5
1017/1017 [==============================] - 138s 135ms/step - loss: 0.9196 - accurac

In [ ]:
generator = Generator( illness = "acl", plane = "sagittal", type1 = "test")
evaluate_acl_sagittal = model6.evaluate_generator(generator=generator)
loss , acc = evaluate_acl_sagittal
print("evaluation done")
print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

evaluation done

Testing loss: 0.055601250380277634, acc: 0.75



In [ ]:
model7 = get_model()
training_generator = Generator( illness = "meniscus", plane = "axial", type1 = "train")
validation_generator = Generator(illness = "meniscus", plane = "axial", type1 = "valid")


# Train model on dataset
history_meniscus_axial = model7.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=False,
                    workers=6, epochs = 5 )
model7.save('/content/gdrive/My Drive/transferVGG_models/model7.h5')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("gl...)`


Model: "model_30"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_28 (InputLayer)        (None, None, 256, 256, 3) 0         
_________________________________________________________________
time_distributed_10 (TimeDis (None, None, 512)         14714688  
_________________________________________________________________
model_29 (Model)             (None, 512)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 513       
Total params: 14,715,201
Trainable params: 513
Non-trainable params: 14,714,688
_________________________________________________________________
Epoch 1/5
1017/1017 [==============================] - 155s 153ms/step - loss: 1.4370 - accuracy: 0.6136 - val_loss: 2.8161 - val_accuracy: 0.6637
Epoch 2/5
1017/1017 [==============================] - 153s 150ms/step - loss: 1.1361 - accurac

In [ ]:
generator = Generator( illness = "meniscus", plane = "axial", type1 = "test")
evaluate_meniscus_axial = model7.evaluate_generator(generator=generator)
loss , acc = evaluate_meniscus_axial
print("evaluation done")
print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

evaluation done

Testing loss: 0.050690341740846634, acc: 0.46666666865348816



In [ ]:
model8 = get_model()
training_generator = Generator( illness = "meniscus", plane = "coronal", type1 = "train")
validation_generator = Generator(illness = "meniscus", plane = "coronal", type1 = "valid")


# Train model on dataset
history_meniscus_coronal = model8.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=False,
                    workers=6, epochs = 5 )
model8.save('/content/gdrive/My Drive/transferVGG_models/model8.h5')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("gl...)`


Model: "model_33"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_31 (InputLayer)        (None, None, 256, 256, 3) 0         
_________________________________________________________________
time_distributed_11 (TimeDis (None, None, 512)         14714688  
_________________________________________________________________
model_32 (Model)             (None, 512)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 513       
Total params: 14,715,201
Trainable params: 513
Non-trainable params: 14,714,688
_________________________________________________________________
Epoch 1/5
1017/1017 [==============================] - 139s 136ms/step - loss: 1.4555 - accuracy: 0.6057 - val_loss: 0.4364 - val_accuracy: 0.6637
Epoch 2/5
1017/1017 [==============================] - 137s 135ms/step - loss: 1.0884 - accurac

In [ ]:
generator = Generator( illness = "meniscus", plane = "coronal", type1 = "test")
evaluate_meniscus_coronal = model8.evaluate_generator(generator=generator)
loss , acc = evaluate_meniscus_coronal
print("evaluation done")
print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

evaluation done

Testing loss: 3.6807541847229004, acc: 0.6666666865348816



In [ ]:
model9 = get_model()
training_generator = Generator( illness = "meniscus", plane = "sagittal", type1 = "train")
validation_generator = Generator(illness = "meniscus", plane = "sagittal", type1 = "valid")


# Train model on dataset
history_meniscus_sagittal = model9.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=False,
                    workers=6, epochs = 5 )
model9.save('/content/gdrive/My Drive/transferVGG_models/model9.h5')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("gl...)`


Model: "model_36"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_34 (InputLayer)        (None, None, 256, 256, 3) 0         
_________________________________________________________________
time_distributed_12 (TimeDis (None, None, 512)         14714688  
_________________________________________________________________
model_35 (Model)             (None, 512)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 513       
Total params: 14,715,201
Trainable params: 513
Non-trainable params: 14,714,688
_________________________________________________________________
Epoch 1/5
1017/1017 [==============================] - 140s 137ms/step - loss: 1.2466 - accuracy: 0.6037 - val_loss: 1.7917 - val_accuracy: 0.6903
Epoch 2/5
1017/1017 [==============================] - 138s 136ms/step - loss: 1.0249 - accurac

In [ ]:
generator = Generator( illness = "meniscus", plane = "sagittal", type1 = "test")
evaluate_meniscus_sagittal = model9.evaluate_generator(generator=generator)
loss , acc = evaluate_meniscus_sagittal
print("evaluation done")
print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

evaluation done

Testing loss: 0.2877332866191864, acc: 0.6916666626930237



In [ ]:
abnormalTestLabels = load_labels('test','abnormal')
aclTestLabels = load_labels('test','acl')
meniscusTestLabels = load_labels('test','meniscus')

abnormalTrainLabels = load_labels('train','abnormal')
aclTrainLabels = load_labels('train','acl')
meniscusTrainLabels = load_labels('train','meniscus')

abnormalValidLabels = load_labels('valid','abnormal')
aclValidLabels = load_labels('valid','acl')
meniscusValidLabels = load_labels('valid','meniscus')

**Predict generator**

In [ ]:
model_abnormal_axial = load_model("/content/gdrive/My Drive/transferVGG_models/model3.h5")
print(model_abnormal_axial)
generator = Generator( illness = "abnormal", plane = "axial", type1 = "test")
predict_abnormal_axial_test = model_abnormal_axial.predict_generator(generator)

generator = Generator( illness = "abnormal", plane = "axial", type1 = "train")
predict_abnormal_axial_train = model_abnormal_axial.predict_generator(generator)

generator = Generator( illness = "abnormal", plane = "axial", type1 = "valid")
predict_abnormal_axial_valid = model_abnormal_axial.predict_generator(generator)

print("prediction done")

prediction done


In [ ]:
model_abnormal_coronal = load_model("/content/gdrive/My Drive/transferVGG_models/model1.h5")
print(model_abnormal_coronal)
generator = Generator( illness = "abnormal", plane = "coronal", type1 = "test")
predict_abnormal_coronal_test = model_abnormal_coronal.predict_generator(generator=generator)

generator = Generator( illness = "abnormal", plane = "coronal", type1 = "train")
predict_abnormal_coronal_train = model_abnormal_coronal.predict_generator(generator=generator)

generator = Generator( illness = "abnormal", plane = "coronal", type1 = "valid")
predict_abnormal_coronal_valid = model_abnormal_coronal.predict_generator(generator=generator)

print("--> prediction done")

--> prediction done


In [ ]:
model_abnormal_sagittal = load_model("/content/gdrive/My Drive/transferVGG_models/model2.h5")
print(model_abnormal_sagittal)
generator = Generator( illness = "abnormal", plane = "sagittal", type1 = "test")
predict_abnormal_sagittal_test = model_abnormal_sagittal.predict_generator(generator=generator)

generator = Generator( illness = "abnormal", plane = "sagittal", type1 = "train")
predict_abnormal_sagittal_train = model_abnormal_sagittal.predict_generator(generator=generator)

generator = Generator( illness = "abnormal", plane = "sagittal", type1 = "valid")
predict_abnormal_sagittal_valid = model_abnormal_sagittal.predict_generator(generator=generator)

print("--> prediction done")

--> prediction done


In [ ]:
model_acl_axial = load_model("/content/gdrive/My Drive/transferVGG_models/model4.h5")
print(model_acl_axial)
generator = Generator( illness = "acl", plane = "axial", type1 = "test")
predict_acl_axial_test = model_acl_axial.predict_generator(generator=generator)

generator = Generator( illness = "acl", plane = "axial", type1 = "train")
predict_acl_axial_train = model_acl_axial.predict_generator(generator=generator)

generator = Generator( illness = "acl", plane = "axial", type1 = "valid")
predict_acl_axial_valid = model_acl_axial.predict_generator(generator=generator)

print("--> prediction done")

--> prediction done


In [ ]:
model_acl_coronal = load_model("/content/gdrive/My Drive/transferVGG_models/model5.h5")
print(model_acl_coronal)
generator = Generator( illness = "acl", plane = "coronal", type1 = "test")
predict_acl_coronal_test = model_acl_coronal.predict_generator(generator=generator)

generator = Generator( illness = "acl", plane = "coronal", type1 = "train")
predict_acl_coronal_train = model_acl_coronal.predict_generator(generator=generator)

generator = Generator( illness = "acl", plane = "coronal", type1 = "valid")
predict_acl_coronal_valid = model_acl_coronal.predict_generator(generator=generator)

print("--> prediction done")

--> prediction done


In [ ]:
model_acl_sagittal = load_model("/content/gdrive/My Drive/transferVGG_models/model6.h5")
print(model_acl_sagittal)
generator = Generator( illness = "acl", plane = "sagittal", type1 = "test")
predict_acl_sagittal_test = model_acl_sagittal.predict_generator(generator=generator)

generator = Generator( illness = "acl", plane = "sagittal", type1 = "train")
predict_acl_sagittal_train = model_acl_sagittal.predict_generator(generator=generator)

generator = Generator( illness = "acl", plane = "sagittal", type1 = "valid")
predict_acl_sagittal_valid = model_acl_sagittal.predict_generator(generator=generator)

print("--> prediction done")

--> prediction done


In [ ]:
model_meniscus_axial = load_model("/content/gdrive/My Drive/transferVGG_models/model7.h5")
print(model_meniscus_axial)
generator = Generator( illness = "meniscus", plane = "axial", type1 = "test")
predict_meniscus_axial_test = model_meniscus_axial.predict_generator(generator=generator)

generator = Generator( illness = "meniscus", plane = "axial", type1 = "train")
predict_meniscus_axial_train = model_meniscus_axial.predict_generator(generator=generator)

generator = Generator( illness = "meniscus", plane = "axial", type1 = "valid")
predict_meniscus_axial_valid = model_meniscus_axial.predict_generator(generator=generator)

print("--> prediction done")

--> prediction done


In [ ]:
model_meniscus_coronal = load_model("/content/gdrive/My Drive/transferVGG_models/model8.h5")
print(model_meniscus_coronal)
generator = Generator( illness = "meniscus", plane = "coronal", type1 = "test")
predict_meniscus_coronal_test = model_meniscus_coronal.predict_generator(generator=generator)

generator = Generator( illness = "meniscus", plane = "coronal", type1 = "train")
predict_meniscus_coronal_train = model_meniscus_coronal.predict_generator(generator=generator)

generator = Generator( illness = "meniscus", plane = "coronal", type1 = "valid")
predict_meniscus_coronal_valid = model_meniscus_coronal.predict_generator(generator=generator)

print("--> prediction done")

--> prediction done


In [ ]:
model_meniscus_sagittal = load_model("/content/gdrive/My Drive/transferVGG_models/model9.h5")
print(model_meniscus_sagittal)
generator = Generator( illness = "meniscus", plane = "sagittal", type1 = "test")
predict_meniscus_sagittal_test = model_meniscus_sagittal.predict_generator(generator=generator)

generator = Generator( illness = "meniscus", plane = "sagittal", type1 = "train")
predict_meniscus_sagittal_train = model_meniscus_sagittal.predict_generator(generator=generator)

generator = Generator( illness = "meniscus", plane = "sagittal", type1 = "valid")
predict_meniscus_sagittal_valid = model_meniscus_sagittal.predict_generator(generator=generator)

print("--> prediction done")

--> prediction done


**Start linear regression**

In [ ]:
Abnormal_model = logistic_regression()

abnormal_train_predictions = []

abnormal_train_predictions.append(predict_abnormal_axial_train)
abnormal_train_predictions.append(predict_abnormal_coronal_train)
abnormal_train_predictions.append(predict_abnormal_sagittal_train)

abnormal_train_predictions = np.transpose(abnormal_train_predictions)

#print(np.array(abnormal_train_predictions)[0].shape)

abnormal_valid_predictions = []
abnormal_valid_predictions.append(predict_abnormal_axial_valid)
abnormal_valid_predictions.append(predict_abnormal_coronal_valid)
abnormal_valid_predictions.append(predict_abnormal_sagittal_valid)

#print(np.array(abnormalValidLabels).shape)

abnormal_valid_predictions = np.transpose(abnormal_valid_predictions)


Abnormal_model.fit( np.array(abnormal_train_predictions)[0] , [abnormalTrainLabels],
                   validation_data = ( np.array(abnormal_valid_predictions)[0] , np.array(abnormalValidLabels)),
                   epochs = 50,use_multiprocessing=True, workers=6)

Abnormal_model.save("/content/gdrive/My Drive/transferVGG_models/Abnormal_model.h5")

Train on 1017 samples, validate on 113 samples
Epoch 1/50
1017/1017 [==============================] - 0s 408us/step - loss: 2.0917 - accuracy: 0.1917 - val_loss: 1.9578 - val_accuracy: 0.1947
Epoch 2/50
1017/1017 [==============================] - 0s 72us/step - loss: 2.0074 - accuracy: 0.1898 - val_loss: 1.8772 - val_accuracy: 0.1947
Epoch 3/50
1017/1017 [==============================] - 0s 71us/step - loss: 1.9250 - accuracy: 0.1858 - val_loss: 1.7986 - val_accuracy: 0.1858
Epoch 4/50
1017/1017 [==============================] - 0s 70us/step - loss: 1.8441 - accuracy: 0.1799 - val_loss: 1.7221 - val_accuracy: 0.1858
Epoch 5/50
1017/1017 [==============================] - 0s 69us/step - loss: 1.7658 - accuracy: 0.1770 - val_loss: 1.6468 - val_accuracy: 0.1858
Epoch 6/50
1017/1017 [==============================] - 0s 83us/step - loss: 1.6886 - accuracy: 0.1711 - val_loss: 1.5749 - val_accuracy: 0.1858
Epoch 7/50
1017/1017 [==============================] - 0s 75us/step - loss: 1.614

In [ ]:
abnormal_test_predictions = []
abnormal_test_predictions.append(predict_abnormal_axial_test)
abnormal_test_predictions.append(predict_abnormal_coronal_test)
abnormal_test_predictions.append(predict_abnormal_sagittal_test)

abnormal_test_predictions = np.transpose(abnormal_test_predictions)


loss1 , acc1 = Abnormal_model.evaluate(abnormal_test_predictions[0] , [abnormalTestLabels])
print("evaluation done")
print('\nTesting loss: {}, acc: {}\n'.format(loss1, acc1))

120/120 [==============================] - 0s 81us/step
evaluation done

Testing loss: 0.5000303079684575, acc: 0.7916666865348816



In [ ]:
ACL_model = logistic_regression()

acl_train_predictions = []
acl_train_predictions.append(predict_acl_axial_train)
acl_train_predictions.append(predict_acl_coronal_train)
acl_train_predictions.append(predict_acl_sagittal_train)

acl_train_predictions = np.transpose(acl_train_predictions)

acl_valid_predictions = []
acl_valid_predictions.append(predict_acl_axial_valid)
acl_valid_predictions.append(predict_acl_coronal_valid)
acl_valid_predictions.append(predict_acl_sagittal_valid)

acl_valid_predictions = np.transpose(acl_valid_predictions)

ACL_model.fit(np.array(acl_train_predictions)[0] , [aclTrainLabels],
                   validation_data = (np.array(acl_valid_predictions)[0], [aclValidLabels]),
                   epochs = 50,use_multiprocessing=True, workers=6)

ACL_model.save("/content/gdrive/My Drive/transferVGG_models/ACL_model.h5")

Train on 1017 samples, validate on 113 samples
Epoch 1/50
1017/1017 [==============================] - 0s 146us/step - loss: 0.6847 - accuracy: 0.8269 - val_loss: 0.6607 - val_accuracy: 0.8584
Epoch 2/50
1017/1017 [==============================] - 0s 75us/step - loss: 0.6743 - accuracy: 0.8220 - val_loss: 0.6492 - val_accuracy: 0.8584
Epoch 3/50
1017/1017 [==============================] - 0s 69us/step - loss: 0.6644 - accuracy: 0.8201 - val_loss: 0.6390 - val_accuracy: 0.8584
Epoch 4/50
1017/1017 [==============================] - 0s 83us/step - loss: 0.6551 - accuracy: 0.8171 - val_loss: 0.6291 - val_accuracy: 0.8584
Epoch 5/50
1017/1017 [==============================] - 0s 76us/step - loss: 0.6462 - accuracy: 0.8161 - val_loss: 0.6197 - val_accuracy: 0.8584
Epoch 6/50
1017/1017 [==============================] - 0s 72us/step - loss: 0.6376 - accuracy: 0.8151 - val_loss: 0.6110 - val_accuracy: 0.8496
Epoch 7/50
1017/1017 [==============================] - 0s 71us/step - loss: 0.629

In [ ]:
acl_test_predictions = []
acl_test_predictions.append(predict_acl_axial_test)
acl_test_predictions.append(predict_acl_coronal_test)
acl_test_predictions.append(predict_acl_sagittal_test)

acl_test_predictions = np.transpose(acl_test_predictions)

loss2, acc2 = ACL_model.evaluate(acl_test_predictions[0], [aclTestLabels])
print("evaluation done")
print('\nTesting loss: {}, acc: {}\n'.format(loss2, acc2))

120/120 [==============================] - 0s 76us/step
evaluation done

Testing loss: 0.6761730670928955, acc: 0.574999988079071



In [ ]:
Meniscus_model = logistic_regression()

meniscus_train_predictions = []
meniscus_train_predictions.append(predict_meniscus_axial_train)
meniscus_train_predictions.append(predict_meniscus_coronal_train)
meniscus_train_predictions.append(predict_meniscus_sagittal_train)

meniscus_train_predictions = np.transpose(meniscus_train_predictions)

meniscus_valid_predictions = []
meniscus_valid_predictions.append(predict_meniscus_axial_valid)
meniscus_valid_predictions.append(predict_meniscus_coronal_valid)
meniscus_valid_predictions.append(predict_meniscus_sagittal_valid)

meniscus_valid_predictions = np.transpose(meniscus_valid_predictions)

Meniscus_model.fit(np.array(meniscus_train_predictions)[0] , [meniscusTrainLabels],
                   validation_data = (np.array(meniscus_valid_predictions)[0], [meniscusValidLabels]),
                   epochs = 50,use_multiprocessing=True, workers=6)

Meniscus_model.save("/content/gdrive/My Drive/transferVGG_models/Meniscus_model.h5")

Train on 1017 samples, validate on 113 samples
Epoch 1/50
1017/1017 [==============================] - 0s 133us/step - loss: 0.6229 - accuracy: 0.6441 - val_loss: 0.6499 - val_accuracy: 0.6283
Epoch 2/50
1017/1017 [==============================] - 0s 71us/step - loss: 0.6183 - accuracy: 0.6775 - val_loss: 0.6479 - val_accuracy: 0.6283
Epoch 3/50
1017/1017 [==============================] - 0s 71us/step - loss: 0.6139 - accuracy: 0.6991 - val_loss: 0.6459 - val_accuracy: 0.6372
Epoch 4/50
1017/1017 [==============================] - 0s 71us/step - loss: 0.6100 - accuracy: 0.7129 - val_loss: 0.6441 - val_accuracy: 0.6372
Epoch 5/50
1017/1017 [==============================] - 0s 70us/step - loss: 0.6058 - accuracy: 0.7158 - val_loss: 0.6421 - val_accuracy: 0.6372
Epoch 6/50
1017/1017 [==============================] - 0s 70us/step - loss: 0.6018 - accuracy: 0.7276 - val_loss: 0.6404 - val_accuracy: 0.6195
Epoch 7/50
1017/1017 [==============================] - 0s 74us/step - loss: 0.598

In [ ]:
meniscus_test_predictions = []
meniscus_test_predictions.append(predict_meniscus_axial_test)
meniscus_test_predictions.append(predict_meniscus_coronal_test)
meniscus_test_predictions.append(predict_meniscus_sagittal_test)
meniscus_test_predictions = np.transpose(meniscus_test_predictions)
loss3, acc3 = Meniscus_model.evaluate(meniscus_test_predictions[0] ,   [meniscusTestLabels])
print("evaluation done")
print('\nTesting loss: {}, acc: {}\n'.format(loss3, acc3))

120/120 [==============================] - 0s 100us/step
evaluation done

Testing loss: 0.5914988954861958, acc: 0.675000011920929



**Get final accurancy and loss**

In [ ]:
print('\nAbnormal Accuracy: {}'.format(acc1))
print('Abnormal Loss    : {}'.format(loss1))
print('\nACL Accuracy     : {}'.format(acc2))
print('ACL Loss         : {}'.format(loss2))
print('\nMeniscus Accuracy: {}\n'.format(acc3))
print('Meniscus Loss    : {}\n'.format(loss3))


Abnormal Accuracy: 0.7916666865348816
Abnormal Loss    : 0.5000303079684575

ACL Accuracy     : 0.574999988079071
ACL Loss         : 0.6761730670928955

Meniscus Accuracy: 0.675000011920929

Meniscus Loss    : 0.5914988954861958

